In [6]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [7]:
pd.set_option('display.max_columns', None)

# Top_1

1. color를 원핫인코딩 변경
2. clothSexCommon이 1인 값은 남자와 여자 모두 1로 변경
3. user테이블에 그 user가 산 옷들 평균내서 fit ~ thickness ~ season까지 저장

In [3]:
top_1 = pd.read_csv('./musinsa_clothes_top_1.csv')

In [4]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [5]:
top_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43356 entries, 0 to 43355
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            43356 non-null  int64  
 1   largeCategory      43356 non-null  int64  
 2   largeCategoryName  43356 non-null  object 
 3   smallCategory      43356 non-null  int64  
 4   smallCategoryName  43356 non-null  object 
 5   colorNo            43356 non-null  int64  
 6   clothName          43356 non-null  object 
 7   brand              43356 non-null  object 
 8   image              43356 non-null  object 
 9   clothPrice         43356 non-null  object 
 10  date               43356 non-null  int64  
 11  hashtags           43356 non-null  object 
 12  clothSexMen        43356 non-null  int64  
 13  clothSexWomen      43356 non-null  int64  
 14  clothSexCommon     43356 non-null  int64  
 15  clothRate          43356 non-null  float64
 16  clothReviewNo      433

In [6]:
top_1.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.335600e+04,43356.0,43356.00000,43356.00000,4.335600e+04,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000,43356.000000
mean,1.777224e+06,1.0,103.68844,5.90239,2.020498e+07,0.705808,0.256228,0.037619,2.287314,1.145678,0.627110,1.054733,1.277816,0.919042,0.212566,0.153335,0.209521,0.147015
std,5.292900e+05,0.0,2.95444,8.85016,1.426893e+04,0.455684,0.436554,0.190275,2.373989,1.890802,1.106783,1.734553,2.039596,1.497869,0.409128,0.360314,0.406972,0.354125
min,1.837900e+04,1.0,101.00000,1.00000,2.012043e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.491044e+06,1.0,101.00000,1.00000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.927994e+06,1.0,103.00000,1.00000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.174280e+06,1.0,105.00000,3.00000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413288e+06,1.0,111.00000,25.00000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
top_1 = top_1.drop_duplicates(subset = "clothId")
top_1 = top_1.reset_index(drop=True)

### smallCategory 원핫인코딩 변환

In [8]:
top_1.insert(4, "smallCategoryOthers", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryShort", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryLong", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryKnit", top_1['smallCategory'], True)
top_1.insert(4, "smallCategorySweatshirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryHoody", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryCollar", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryShirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryHalfshort", top_1['smallCategory'], True)

In [9]:
top_1['smallCategoryHalfshort'] = top_1['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_1['smallCategoryShirt'] = top_1['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_1['smallCategoryCollar'] = top_1['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_1['smallCategoryHoody'] = top_1['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_1['smallCategorySweatshirt'] = top_1['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_1['smallCategoryKnit'] = top_1['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_1['smallCategoryLong'] = top_1['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_1['smallCategoryShort'] = top_1['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_1['smallCategoryOthers'] = top_1['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

In [10]:
top_1['largeCategory'].unique()

array([1], dtype=int64)

In [11]:
top_1['largeCategoryName'].unique()

array(['상의'], dtype=object)

In [12]:
top_1['smallCategory'].unique()

array([101, 103, 111, 110, 108, 105, 106, 102, 104], dtype=int64)

In [13]:
top_1['smallCategoryName'].unique()

array(['반팔 티셔츠', '피케/카라 티셔츠', '민소매 티셔츠', '긴팔 티셔츠', '기타 상의', '맨투맨/스웨트셔츠',
       '니트/스웨터', '셔츠/블라우스', '후드 티셔츠'], dtype=object)

In [14]:
top_1 = top_1.drop('smallCategory', axis=1)

In [15]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [16]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_1[top_1.columns[i]].value_counts()[1]
print(num)
print(top_1.shape)

43255
(43255, 34)


### clothSexCommon 변환

In [17]:
top_1.head()

,clothId,largeCategory,largeCategoryName,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,"59,000원",20210429,[],1,0,0,0.0,0,0,0,0,0,0,0,0,0,0
1,1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,0,0,0.0,0,0,0,0,0,0,0,0,0,0
2,1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,"43,000원",20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,0,0,4.5,2,0,0,0,0,0,0,0,0,0
3,1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,"26,600원",20210429,[],0,1,0,5.0,1,0,0,0,0,0,0,0,0,0
4,1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,"24,000원",20210429,[],1,0,0,0.0,0,0,0,0,0,0,0,0,0,0


In [18]:
top_1['clothSexMen'] = top_1.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_1['clothSexWomen'] = top_1.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [19]:
top_1 = top_1.drop('clothSexCommon', axis=1)

In [20]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [21]:
top_1['clothSexMen'] = top_1.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_1['clothSexWomen'] = top_1.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [22]:
print(top_1['clothSexMen'].value_counts())
print(top_1['clothSexWomen'].value_counts())
print(top_1.shape)

1    32161
0    11094
Name: clothSexMen, dtype: int64
0    30530
1    12725
Name: clothSexWomen, dtype: int64
(43255, 33)


### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [23]:
top_1.insert(14, "colorOthers", top_1['colorNo'], True)
top_1.insert(14, "colorPattern", top_1['colorNo'], True)
top_1.insert(14, "colorJean", top_1['colorNo'], True)
top_1.insert(14, "colorBeige", top_1['colorNo'], True)
top_1.insert(14, "colorBrown", top_1['colorNo'], True)
top_1.insert(14, "colorPurple", top_1['colorNo'], True)
top_1.insert(14, "colorBlue", top_1['colorNo'], True)
top_1.insert(14, "colorGreen", top_1['colorNo'], True)
top_1.insert(14, "colorYellow", top_1['colorNo'], True)
top_1.insert(14, "colorIvory", top_1['colorNo'], True)
top_1.insert(14, "colorOrange", top_1['colorNo'], True)
top_1.insert(14, "colorPink", top_1['colorNo'], True)
top_1.insert(14, "colorRed", top_1['colorNo'], True)
top_1.insert(14, "colorBlack", top_1['colorNo'], True)
top_1.insert(14, "colorGrey", top_1['colorNo'], True)
top_1.insert(14, "colorWhite", top_1['colorNo'], True)

In [24]:
top_1['colorWhite'] = top_1['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_1['colorGrey'] = top_1['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_1['colorBlack'] = top_1['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_1['colorRed'] = top_1['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_1['colorPink'] = top_1['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_1['colorOrange'] = top_1['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_1['colorIvory'] = top_1['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_1['colorYellow'] = top_1['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_1['colorGreen'] = top_1['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_1['colorBlue'] = top_1['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_1['colorPurple'] = top_1['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_1['colorBrown'] = top_1['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_1['colorBeige'] = top_1['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_1['colorJean'] = top_1['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_1['colorPattern'] = top_1['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_1['colorOthers'] = top_1['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [25]:
top_1 = top_1.drop('colorNo', axis=1)

In [26]:
top_1.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [27]:
top_1.insert(29, "colorName", True)

In [28]:
top_1['colorName'] = top_1.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [29]:
top_1['colorName'].value_counts()

white    22658
grey     20597
Name: colorName, dtype: int64

### Price 전처리

In [30]:
top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace(",", ""))
top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace("원", ""))
top_1['clothPrice']

0        59000
1        43000
2        43000
3        26600
4        24000
         ...  
43250    63200
43251    69000
43252    32200
43253    79000
43254    26900
Name: clothPrice, Length: 43255, dtype: object

In [31]:
top_1['clothPrice'] = top_1['clothPrice'].astype('int')

In [32]:
top_1['clothReviewNo'] = top_1['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_1['clothReviewNo'] = top_1['clothReviewNo'].astype('int')
top_1['clothReviewNo']

0         0
1         0
2         2
3         1
4         0
         ..
43250     0
43251     0
43252     7
43253     0
43254    14
Name: clothReviewNo, Length: 43255, dtype: int32

In [33]:
top_1.index = top_1['clothId']
top_1 = top_1.drop('clothId', True)
print(top_1)

         largeCategory largeCategoryName  smallCategoryHalfshort  \
clothId                                                            
1929966              1                상의                       1   
1927658              1                상의                       1   
1927660              1                상의                       1   
1928209              1                상의                       1   
1927270              1                상의                       1   
...                ...               ...                     ...   
2346546              1                상의                       0   
2347944              1                상의                       0   
2239875              1                상의                       0   
2239251              1                상의                       0   
1276479              1                상의                       0   

         smallCategoryShirt  smallCategoryCollar  smallCategoryHoody  \
clothId                                    

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\2457559906.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  top_1 = top_1.drop('clothId', True)


In [34]:
top_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43255 entries, 1929966 to 1276479
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   largeCategory            43255 non-null  int64  
 1   largeCategoryName        43255 non-null  object 
 2   smallCategoryHalfshort   43255 non-null  int64  
 3   smallCategoryShirt       43255 non-null  int64  
 4   smallCategoryCollar      43255 non-null  int64  
 5   smallCategoryHoody       43255 non-null  int64  
 6   smallCategorySweatshirt  43255 non-null  int64  
 7   smallCategoryKnit        43255 non-null  int64  
 8   smallCategoryLong        43255 non-null  int64  
 9   smallCategoryShort       43255 non-null  int64  
 10  smallCategoryOthers      43255 non-null  int64  
 11  smallCategoryName        43255 non-null  object 
 12  colorWhite               43255 non-null  int64  
 13  colorGrey                43255 non-null  int64  
 14  colorBlack    

# Top_2

In [35]:
top_2 = pd.read_csv('./musinsa_clothes_top_2.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\3913089728.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_2 = pd.read_csv('./musinsa_clothes_top_2.csv')


In [36]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [37]:
top_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41149 entries, 0 to 41148
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            41149 non-null  int64  
 1   largeCategory      41149 non-null  int64  
 2   largeCategoryName  41149 non-null  object 
 3   smallCategory      41149 non-null  object 
 4   smallCategoryName  41149 non-null  object 
 5   colorNo            41149 non-null  int64  
 6   clothName          41149 non-null  object 
 7   brand              41149 non-null  object 
 8   image              41149 non-null  object 
 9   clothPrice         41149 non-null  object 
 10  date               41149 non-null  int64  
 11  hashtags           41149 non-null  object 
 12  clothSexMen        41149 non-null  int64  
 13  clothSexWomen      41149 non-null  int64  
 14  clothSexCommon     41149 non-null  int64  
 15  clothRate          41149 non-null  float64
 16  clothReviewNo      411

In [38]:
top_2.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,4.114900e+04,41149.0,41149.000000,4.114900e+04,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000,41149.000000
mean,1.737427e+06,1.0,3.439209,2.020381e+07,0.710418,0.253931,0.035359,2.174320,1.006100,0.583100,0.961846,1.204355,0.846339,0.184306,0.127172,0.185910,0.135046
std,5.569147e+05,0.0,5.317300,1.508345e+04,0.453574,0.435264,0.184689,2.367866,1.799973,1.091305,1.683685,2.034208,1.475501,0.387738,0.333170,0.389039,0.341777
min,1.039600e+04,1.0,2.000000,2.011061e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.411780e+06,1.0,2.000000,2.020042e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.905913e+06,1.0,2.000000,2.021042e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.178246e+06,1.0,2.000000,2.021101e+07,1.000000,1.000000,0.000000,4.800000,2.000000,1.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,2.413516e+06,1.0,51.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
top_2 = top_2.drop_duplicates(subset = "clothId")
top_2 = top_2.reset_index(drop=True)

In [40]:
top_2['smallCategory'] = top_2['smallCategory'].astype('str')

In [41]:
top_2['largeCategory'].unique()

array([1], dtype=int64)

In [42]:
top_2['largeCategoryName'].unique()

array(['상의'], dtype=object)

In [43]:
top_2['smallCategory'].unique()

array(['103', '110', '106', '105', '101', '108', '104', '102', '111',
       '1s='], dtype=object)

In [44]:
top_2['smallCategoryName'].unique()

array(['피케/카라 티셔츠', '긴팔 티셔츠', '니트/스웨터', '맨투맨/스웨트셔츠', '반팔 티셔츠', '기타 상의',
       '후드 티셔츠', '셔츠/블라우스', '민소매 티셔츠', '\r\n            (나이키)        '],
      dtype=object)

In [45]:
top_2[top_2['smallCategoryName']=='\r\n            (나이키)        ']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
31991,1385204,1,상의,1s=,\r\n (나이키),2,반팔티 레전드 2.0 NI-TT7 택1 718833,나이키,//image.msscdn.net/images/goods_img/20200403/1...,"38,900원",20200403,[],1,0,0,5.0,1,0,0,0,0,0,0,0,0,0


In [46]:
top_2.loc[top_2.smallCategoryName=='\r\n            (나이키)        ', ('smallCategoryName')] = '반팔 티셔츠'

In [47]:
top_2[top_2['smallCategory']=="1s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
31991,1385204,1,상의,1s=,반팔 티셔츠,2,반팔티 레전드 2.0 NI-TT7 택1 718833,나이키,//image.msscdn.net/images/goods_img/20200403/1...,"38,900원",20200403,[],1,0,0,5.0,1,0,0,0,0,0,0,0,0,0


In [48]:
top_2.loc[top_2.smallCategory=='1s=', ('smallCategory')] = '101'

In [49]:
top_2['smallCategory'] = top_2['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [50]:
top_2.insert(4, "smallCategoryOthers", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryShort", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryLong", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryKnit", top_2['smallCategory'], True)
top_2.insert(4, "smallCategorySweatshirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryHoody", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryCollar", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryShirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryHalfshort", top_2['smallCategory'], True)

In [51]:
top_2['smallCategoryHalfshort'] = top_2['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_2['smallCategoryShirt'] = top_2['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_2['smallCategoryCollar'] = top_2['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_2['smallCategoryHoody'] = top_2['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_2['smallCategorySweatshirt'] = top_2['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_2['smallCategoryKnit'] = top_2['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_2['smallCategoryLong'] = top_2['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_2['smallCategoryShort'] = top_2['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_2['smallCategoryOthers'] = top_2['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

In [52]:
top_2 = top_2.drop('smallCategory', axis=1)

In [53]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [54]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_2[top_2.columns[i]].value_counts()[1]
print(num)
print(top_2.shape)

40985
(40985, 34)


### clothSexCommon 변환

In [55]:
top_2['clothSexMen'] = top_2.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_2['clothSexWomen'] = top_2.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [56]:
top_2 = top_2.drop('clothSexCommon', axis=1)

In [57]:
top_2.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [58]:
top_2['clothSexMen'] = top_2.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_2['clothSexWomen'] = top_2.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [59]:
print(top_2['clothSexMen'].value_counts())
print(top_2['clothSexWomen'].value_counts())
print(top_2.shape)

1    30580
0    10405
Name: clothSexMen, dtype: int64
0    29133
1    11852
Name: clothSexWomen, dtype: int64
(40985, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [60]:
top_2.insert(14, "colorOthers", top_2['colorNo'], True)
top_2.insert(14, "colorPattern", top_2['colorNo'], True)
top_2.insert(14, "colorJean", top_2['colorNo'], True)
top_2.insert(14, "colorBeige", top_2['colorNo'], True)
top_2.insert(14, "colorBrown", top_2['colorNo'], True)
top_2.insert(14, "colorPurple", top_2['colorNo'], True)
top_2.insert(14, "colorBlue", top_2['colorNo'], True)
top_2.insert(14, "colorGreen", top_2['colorNo'], True)
top_2.insert(14, "colorYellow", top_2['colorNo'], True)
top_2.insert(14, "colorIvory", top_2['colorNo'], True)
top_2.insert(14, "colorOrange", top_2['colorNo'], True)
top_2.insert(14, "colorPink", top_2['colorNo'], True)
top_2.insert(14, "colorRed", top_2['colorNo'], True)
top_2.insert(14, "colorBlack", top_2['colorNo'], True)
top_2.insert(14, "colorGrey", top_2['colorNo'], True)
top_2.insert(14, "colorWhite", top_2['colorNo'], True)

In [61]:
top_2['colorWhite'] = top_2['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_2['colorGrey'] = top_2['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_2['colorBlack'] = top_2['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_2['colorRed'] = top_2['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_2['colorPink'] = top_2['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_2['colorOrange'] = top_2['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_2['colorIvory'] = top_2['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_2['colorYellow'] = top_2['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_2['colorGreen'] = top_2['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_2['colorBlue'] = top_2['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_2['colorPurple'] = top_2['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_2['colorBrown'] = top_2['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_2['colorBeige'] = top_2['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_2['colorJean'] = top_2['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_2['colorPattern'] = top_2['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_2['colorOthers'] = top_2['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [62]:
top_2 = top_2.drop('colorNo', axis=1)

In [63]:
print(top_2.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [64]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_2[top_2.columns[i]].value_counts())
print(top_2.shape)

0    40985
Name: colorWhite, dtype: int64
0    40985
Name: colorGrey, dtype: int64
1    35846
0     5139
Name: colorBlack, dtype: int64
0    38313
1     2672
Name: colorRed, dtype: int64
0    38518
1     2467
Name: colorPink, dtype: int64
0    40985
Name: colorOrange, dtype: int64
0    40985
Name: colorIvory, dtype: int64
0    40985
Name: colorYellow, dtype: int64
0    40985
Name: colorGreen, dtype: int64
0    40985
Name: colorBlue, dtype: int64
0    40985
Name: colorPurple, dtype: int64
0    40985
Name: colorBrown, dtype: int64
0    40985
Name: colorBeige, dtype: int64
0    40985
Name: colorJean, dtype: int64
0    40985
Name: colorPattern, dtype: int64
0    40985
Name: colorOthers, dtype: int64
(40985, 48)


In [65]:
top_2.insert(29, "colorName", True)

In [66]:
top_2['colorName'] = top_2.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [67]:
top_2['colorName'].value_counts()

black    35846
red       2672
pink      2467
Name: colorName, dtype: int64

### Price 전처리

In [68]:
top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace(",", ""))
top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace("원", ""))
top_2['clothPrice']

0        109000
1         99000
2         69000
3        168000
4        185000
          ...  
40980     66500
40981     83300
40982     66500
40983     83300
40984     29900
Name: clothPrice, Length: 40985, dtype: object

In [69]:
top_2['clothPrice'] = top_2['clothPrice'].astype('int')

In [70]:
top_2['clothReviewNo'] = top_2['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_2['clothReviewNo'] = top_2['clothReviewNo'].astype('int')
top_2['clothReviewNo']

0        0
1        0
2        2
3        0
4        0
        ..
40980    0
40981    0
40982    0
40983    0
40984    0
Name: clothReviewNo, Length: 40985, dtype: int32

In [71]:
top_2.index = top_2['clothId']
top_2 = top_2.drop('clothId', True)
print(top_2)

         largeCategory largeCategoryName  smallCategoryHalfshort  \
clothId                                                            
2388725              1                상의                       0   
2388714              1                상의                       0   
2017016              1                상의                       0   
2385232              1                상의                       0   
2385223              1                상의                       0   
...                ...               ...                     ...   
2336337              1                상의                       0   
2336234              1                상의                       0   
2336240              1                상의                       0   
2336324              1                상의                       0   
2008157              1                상의                       0   

         smallCategoryShirt  smallCategoryCollar  smallCategoryHoody  \
clothId                                    

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\2025520806.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  top_2 = top_2.drop('clothId', True)


In [72]:
top_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40985 entries, 2388725 to 2008157
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   largeCategory            40985 non-null  int64  
 1   largeCategoryName        40985 non-null  object 
 2   smallCategoryHalfshort   40985 non-null  int64  
 3   smallCategoryShirt       40985 non-null  int64  
 4   smallCategoryCollar      40985 non-null  int64  
 5   smallCategoryHoody       40985 non-null  int64  
 6   smallCategorySweatshirt  40985 non-null  int64  
 7   smallCategoryKnit        40985 non-null  int64  
 8   smallCategoryLong        40985 non-null  int64  
 9   smallCategoryShort       40985 non-null  int64  
 10  smallCategoryOthers      40985 non-null  int64  
 11  smallCategoryName        40985 non-null  object 
 12  colorWhite               40985 non-null  int64  
 13  colorGrey                40985 non-null  int64  
 14  colorBlack    

# Top_3

In [73]:
top_3 = pd.read_csv('./musinsa_clothes_top_3.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\3558435819.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_3 = pd.read_csv('./musinsa_clothes_top_3.csv')


In [74]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [75]:
top_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39401 entries, 0 to 39400
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            39401 non-null  int64  
 1   largeCategory      39401 non-null  int64  
 2   largeCategoryName  39401 non-null  object 
 3   smallCategory      39401 non-null  object 
 4   smallCategoryName  39401 non-null  object 
 5   colorNo            39401 non-null  int64  
 6   clothName          39401 non-null  object 
 7   brand              39401 non-null  object 
 8   image              39401 non-null  object 
 9   clothPrice         39401 non-null  object 
 10  date               39401 non-null  int64  
 11  hashtags           39401 non-null  object 
 12  clothSexMen        39401 non-null  int64  
 13  clothSexWomen      39401 non-null  int64  
 14  clothSexCommon     39401 non-null  int64  
 15  clothRate          39401 non-null  float64
 16  clothReviewNo      394

In [76]:
top_3.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,3.940100e+04,39401.0,39401.000000,3.940100e+04,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000,39401.000000
mean,1.805844e+06,1.0,22.145453,2.020617e+07,0.616583,0.340347,0.042842,2.412261,1.136773,0.646024,1.065658,1.335829,0.934164,0.215375,0.135555,0.216390,0.150707
std,4.787596e+05,0.0,14.032963,1.189465e+04,0.486225,0.473832,0.202502,2.371745,1.853368,1.120235,1.737210,2.088987,1.500704,0.411088,0.342320,0.411789,0.357767
min,1.039000e+04,1.0,6.000000,2.011061e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.493499e+06,1.0,7.000000,2.020062e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.919152e+06,1.0,23.000000,2.021042e+07,1.000000,0.000000,0.000000,3.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.183692e+06,1.0,33.000000,2.021102e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413604e+06,1.0,55.000000,2.022031e+07,1.000000,1.000000,1.000000,6.700000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
top_3 = top_3.drop_duplicates(subset = "clothId")
top_3 = top_3.reset_index(drop=True)

In [78]:
top_3['smallCategory'] = top_3['smallCategory'].astype('str')

In [79]:
top_3['largeCategory'].unique()

array([1], dtype=int64)

In [80]:
top_3['largeCategoryName'].unique()

array(['상의'], dtype=object)

In [81]:
top_3['smallCategory'].unique()

array(['104', '101', '105', '103', '106', '111', '102', '110', '108',
       '1s='], dtype=object)

In [82]:
top_3['smallCategoryName'].unique()

array(['후드 티셔츠', '반팔 티셔츠', '맨투맨/스웨트셔츠', '피케/카라 티셔츠', '니트/스웨터', '민소매 티셔츠',
       '셔츠/블라우스', '긴팔 티셔츠', '기타 상의', '\r\n            (로라로라)        '],
      dtype=object)

In [83]:
top_3[top_3['smallCategoryName']=='\r\n            (나이키)        ']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter


In [84]:
top_3.loc[top_3.smallCategoryName=='\r\n            (로라로라)        ', ('smallCategoryName')] = '후드 티셔츠'

In [85]:
top_3[top_3['smallCategory']=="1s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
18797,1541525,1,상의,1s=,후드 티셔츠,32,(HD-20521) ROLA SIGNATURE HOOD T-SHIRT MINT,로라로라,//image.msscdn.net/images/goods_img/20200807/1...,"63,200원",20200807,"['시그니처후드', '로고후드', '파스텔후드', '자수후드', '후드티', '로고...",0,1,0,4.7,108,0,0,0,0,0,0,0,0,0


In [86]:
top_3.loc[top_3.smallCategory=='1s=', ('smallCategory')] = '104'

In [87]:
top_3['smallCategory'] = top_3['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [88]:
top_3.insert(4, "smallCategoryOthers", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryShort", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryLong", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryKnit", top_3['smallCategory'], True)
top_3.insert(4, "smallCategorySweatshirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryHoody", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryCollar", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryShirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryHalfshort", top_3['smallCategory'], True)

In [89]:
top_3['smallCategoryHalfshort'] = top_3['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_3['smallCategoryShirt'] = top_3['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_3['smallCategoryCollar'] = top_3['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_3['smallCategoryHoody'] = top_3['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_3['smallCategorySweatshirt'] = top_3['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_3['smallCategoryKnit'] = top_3['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_3['smallCategoryLong'] = top_3['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_3['smallCategoryShort'] = top_3['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_3['smallCategoryOthers'] = top_3['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

In [90]:
top_3 = top_3.drop('smallCategory', axis=1)

In [91]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [92]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_3[top_3.columns[i]].value_counts()[1]
    print(top_3[top_3.columns[i]].value_counts())
print(num)
print(top_3.shape)

0    28443
1    10926
Name: smallCategoryHalfshort, dtype: int64
0    33283
1     6086
Name: smallCategoryShirt, dtype: int64
0    38020
1     1349
Name: smallCategoryCollar, dtype: int64
0    35343
1     4026
Name: smallCategoryHoody, dtype: int64
0    31467
1     7902
Name: smallCategorySweatshirt, dtype: int64
0    34745
1     4624
Name: smallCategoryKnit, dtype: int64
0    35901
1     3468
Name: smallCategoryLong, dtype: int64
0    38753
1      616
Name: smallCategoryShort, dtype: int64
0    38997
1      372
Name: smallCategoryOthers, dtype: int64
39369
(39369, 34)


### clothSexCommon 변환

In [93]:
top_3['clothSexMen'] = top_3.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_3['clothSexWomen'] = top_3.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [94]:
top_3 = top_3.drop('clothSexCommon', axis=1)

In [95]:
top_3.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [96]:
top_3['clothSexMen'] = top_3.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_3['clothSexWomen'] = top_3.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [97]:
print(top_3['clothSexMen'].value_counts())
print(top_3['clothSexWomen'].value_counts())
print(top_3.shape)

1    25970
0    13399
Name: clothSexMen, dtype: int64
0    24282
1    15087
Name: clothSexWomen, dtype: int64
(39369, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [98]:
top_3.insert(14, "colorOthers", top_3['colorNo'], True)
top_3.insert(14, "colorPattern", top_3['colorNo'], True)
top_3.insert(14, "colorJean", top_3['colorNo'], True)
top_3.insert(14, "colorBeige", top_3['colorNo'], True)
top_3.insert(14, "colorBrown", top_3['colorNo'], True)
top_3.insert(14, "colorPurple", top_3['colorNo'], True)
top_3.insert(14, "colorBlue", top_3['colorNo'], True)
top_3.insert(14, "colorGreen", top_3['colorNo'], True)
top_3.insert(14, "colorYellow", top_3['colorNo'], True)
top_3.insert(14, "colorIvory", top_3['colorNo'], True)
top_3.insert(14, "colorOrange", top_3['colorNo'], True)
top_3.insert(14, "colorPink", top_3['colorNo'], True)
top_3.insert(14, "colorRed", top_3['colorNo'], True)
top_3.insert(14, "colorBlack", top_3['colorNo'], True)
top_3.insert(14, "colorGrey", top_3['colorNo'], True)
top_3.insert(14, "colorWhite", top_3['colorNo'], True)

In [99]:
top_3['colorWhite'] = top_3['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_3['colorGrey'] = top_3['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_3['colorBlack'] = top_3['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_3['colorRed'] = top_3['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_3['colorPink'] = top_3['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_3['colorOrange'] = top_3['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_3['colorIvory'] = top_3['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_3['colorYellow'] = top_3['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_3['colorGreen'] = top_3['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_3['colorBlue'] = top_3['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_3['colorPurple'] = top_3['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_3['colorBrown'] = top_3['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_3['colorBeige'] = top_3['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_3['colorJean'] = top_3['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_3['colorPattern'] = top_3['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_3['colorOthers'] = top_3['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [100]:
print(top_3.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [101]:
top_3 = top_3.drop('colorNo', axis=1)

In [102]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_3[top_3.columns[i]].value_counts())
print(top_3.shape)

0    39369
Name: colorWhite, dtype: int64
0    39369
Name: colorGrey, dtype: int64
0    39369
Name: colorBlack, dtype: int64
0    39369
Name: colorRed, dtype: int64
0    37611
1     1758
Name: colorPink, dtype: int64
0    36857
1     2512
Name: colorOrange, dtype: int64
0    31608
1     7761
Name: colorIvory, dtype: int64
0    35413
1     3956
Name: colorYellow, dtype: int64
0    27512
1    11857
Name: colorGreen, dtype: int64
0    27844
1    11525
Name: colorBlue, dtype: int64
0    39369
Name: colorPurple, dtype: int64
0    39369
Name: colorBrown, dtype: int64
0    39369
Name: colorBeige, dtype: int64
0    39369
Name: colorJean, dtype: int64
0    39369
Name: colorPattern, dtype: int64
0    39369
Name: colorOthers, dtype: int64
(39369, 48)


In [103]:
top_3.insert(29, "colorName", True)

In [104]:
top_3['colorName'] = top_3.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [105]:
top_3['colorName'].value_counts()

green     11857
blue      11525
ivory      7761
yellow     3956
orange     2512
pink       1758
Name: colorName, dtype: int64

### Price 전처리

In [106]:
top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace(",", ""))
top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace("원", ""))
top_3['clothPrice']

0        36000
1        25200
2        34500
3        35000
4        29000
         ...  
39364    39000
39365    39000
39366    62300
39367    59000
39368    49000
Name: clothPrice, Length: 39369, dtype: object

In [107]:
top_3['clothPrice'] = top_3['clothPrice'].astype('int')

In [108]:
top_3['clothReviewNo'] = top_3['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_3['clothReviewNo'] = top_3['clothReviewNo'].astype('int')
top_3['clothReviewNo']

0        0
1        0
2        0
3        0
4        8
        ..
39364    0
39365    1
39366    0
39367    0
39368    0
Name: clothReviewNo, Length: 39369, dtype: int32

In [109]:
top_3.index = top_3['clothId']
top_3 = top_3.drop('clothId', True)
print(top_3)

         largeCategory largeCategoryName  smallCategoryHalfshort  \
clothId                                                            
1650399              1                상의                       0   
1616358              1                상의                       0   
1624441              1                상의                       0   
1545028              1                상의                       1   
1284050              1                상의                       0   
...                ...               ...                     ...   
2047516              1                상의                       1   
2047520              1                상의                       1   
2047672              1                상의                       1   
2041444              1                상의                       1   
2041003              1                상의                       1   

         smallCategoryShirt  smallCategoryCollar  smallCategoryHoody  \
clothId                                    

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\4209233628.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  top_3 = top_3.drop('clothId', True)


In [110]:
top_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39369 entries, 1650399 to 2041003
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   largeCategory            39369 non-null  int64  
 1   largeCategoryName        39369 non-null  object 
 2   smallCategoryHalfshort   39369 non-null  int64  
 3   smallCategoryShirt       39369 non-null  int64  
 4   smallCategoryCollar      39369 non-null  int64  
 5   smallCategoryHoody       39369 non-null  int64  
 6   smallCategorySweatshirt  39369 non-null  int64  
 7   smallCategoryKnit        39369 non-null  int64  
 8   smallCategoryLong        39369 non-null  int64  
 9   smallCategoryShort       39369 non-null  int64  
 10  smallCategoryOthers      39369 non-null  int64  
 11  smallCategoryName        39369 non-null  object 
 12  colorWhite               39369 non-null  int64  
 13  colorGrey                39369 non-null  int64  
 14  colorBlack    

# Top_4

In [111]:
top_4 = pd.read_csv('./musinsa_clothes_top_4.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\4024622213.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_4 = pd.read_csv('./musinsa_clothes_top_4.csv')


In [112]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [113]:
top_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34326 entries, 0 to 34325
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            34326 non-null  int64  
 1   largeCategory      34326 non-null  int64  
 2   largeCategoryName  34326 non-null  object 
 3   smallCategory      34326 non-null  object 
 4   smallCategoryName  34326 non-null  object 
 5   colorNo            34326 non-null  int64  
 6   clothName          34326 non-null  object 
 7   brand              34326 non-null  object 
 8   image              34326 non-null  object 
 9   clothPrice         34326 non-null  object 
 10  date               34326 non-null  int64  
 11  hashtags           34326 non-null  object 
 12  clothSexMen        34326 non-null  int64  
 13  clothSexWomen      34326 non-null  int64  
 14  clothSexCommon     34326 non-null  int64  
 15  clothRate          34326 non-null  float64
 16  clothReviewNo      343

In [114]:
top_4.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,3.432600e+04,34326.0,34326.000000,3.432600e+04,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000,34326.000000
mean,1.820122e+06,1.0,21.886209,2.020614e+07,0.663142,0.298112,0.038513,2.427746,1.084251,0.611111,0.986016,1.273058,0.893142,0.206666,0.117083,0.215027,0.159325
std,4.854657e+05,0.0,14.556563,1.256095e+04,0.472643,0.457436,0.192434,2.374646,1.828386,1.099026,1.686770,2.073083,1.494306,0.404919,0.321524,0.410847,0.365984
min,1.765900e+04,1.0,4.000000,2.012041e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.553120e+06,1.0,5.000000,2.020082e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.941796e+06,1.0,19.000000,2.021051e+07,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.186231e+06,1.0,36.000000,2.021102e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,2.413286e+06,1.0,71.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,5.000000,6.000000,1.000000,1.000000,1.000000,1.000000


In [115]:
top_4 = top_4.drop_duplicates(subset = "clothId")
top_4 = top_4.reset_index(drop=True)

In [116]:
top_4['smallCategory'] = top_4['smallCategory'].astype('str')

In [117]:
top_4['largeCategory'].unique()

array([1], dtype=int64)

In [118]:
top_4['largeCategoryName'].unique()

array(['상의'], dtype=object)

In [119]:
top_4['smallCategory'].unique()

array(['104', '106', '110', '103', '101', '105', '102', '108', '111',
       '1s='], dtype=object)

In [120]:
top_4['smallCategoryName'].unique()

array(['후드 티셔츠', '니트/스웨터', '긴팔 티셔츠', '피케/카라 티셔츠', '반팔 티셔츠', '맨투맨/스웨트셔츠',
       '셔츠/블라우스', '기타 상의', '민소매 티셔츠', '\n            (로라로라)        '],
      dtype=object)

In [121]:
top_4[top_4['smallCategoryName']=='\n            (로라로라)        ']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
13150,1541526,1,상의,1s=,\n (로라로라),39,(HD-20521) ROLA SIGNATURE HOOD T-SHIRT LAVENDER,로라로라,//image.msscdn.net/images/goods_img/20200807/1...,"63,200원",20200807,"['시그니처후드', '로고후드', '파스텔후드', '자수후드', '후드티', '로고...",0,1,0,4.7,108,0,0,0,0,0,0,0,0,0


In [122]:
top_4.loc[top_4.smallCategoryName=='\n            (로라로라)        ', ('smallCategoryName')] = '후드 티셔츠'

In [123]:
top_4[top_4['smallCategory']=="1s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
13150,1541526,1,상의,1s=,후드 티셔츠,39,(HD-20521) ROLA SIGNATURE HOOD T-SHIRT LAVENDER,로라로라,//image.msscdn.net/images/goods_img/20200807/1...,"63,200원",20200807,"['시그니처후드', '로고후드', '파스텔후드', '자수후드', '후드티', '로고...",0,1,0,4.7,108,0,0,0,0,0,0,0,0,0


In [124]:
top_4.loc[top_4.smallCategory=='1s=', ('smallCategory')] = '104'

In [125]:
top_4['smallCategory'] = top_4['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [126]:
top_4.insert(4, "smallCategoryOthers", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryShort", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryLong", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryKnit", top_4['smallCategory'], True)
top_4.insert(4, "smallCategorySweatshirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryHoody", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryCollar", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryShirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryHalfshort", top_4['smallCategory'], True)

In [127]:
top_4['smallCategoryHalfshort'] = top_4['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_4['smallCategoryShirt'] = top_4['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_4['smallCategoryCollar'] = top_4['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_4['smallCategoryHoody'] = top_4['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_4['smallCategorySweatshirt'] = top_4['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_4['smallCategoryKnit'] = top_4['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_4['smallCategoryLong'] = top_4['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_4['smallCategoryShort'] = top_4['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_4['smallCategoryOthers'] = top_4['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

In [128]:
top_4 = top_4.drop('smallCategory', axis=1)

In [129]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [130]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_4[top_4.columns[i]].value_counts()[1]
    print(top_4[top_4.columns[i]].value_counts())
print(num)
print(top_4.shape)

0    26345
1     7935
Name: smallCategoryHalfshort, dtype: int64
0    27629
1     6651
Name: smallCategoryShirt, dtype: int64
0    32938
1     1342
Name: smallCategoryCollar, dtype: int64
0    30988
1     3292
Name: smallCategoryHoody, dtype: int64
0    27859
1     6421
Name: smallCategorySweatshirt, dtype: int64
0    29814
1     4466
Name: smallCategoryKnit, dtype: int64
0    31086
1     3194
Name: smallCategoryLong, dtype: int64
0    33746
1      534
Name: smallCategoryShort, dtype: int64
0    33835
1      445
Name: smallCategoryOthers, dtype: int64
34280
(34280, 34)


### clothSexCommon 변환

In [131]:
top_4['clothSexMen'] = top_4.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_4['clothSexWomen'] = top_4.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [132]:
top_4 = top_4.drop('clothSexCommon', axis=1)

In [133]:
top_4.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [134]:
top_4['clothSexMen'] = top_4.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_4['clothSexWomen'] = top_4.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [135]:
print(top_4['clothSexMen'].value_counts())
print(top_4['clothSexWomen'].value_counts())
print(top_4.shape)

1    24057
0    10223
Name: clothSexMen, dtype: int64
0    22735
1    11545
Name: clothSexWomen, dtype: int64
(34280, 33)


### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [136]:
top_4.insert(14, "colorOthers", top_4['colorNo'], True)
top_4.insert(14, "colorPattern", top_4['colorNo'], True)
top_4.insert(14, "colorJean", top_4['colorNo'], True)
top_4.insert(14, "colorBeige", top_4['colorNo'], True)
top_4.insert(14, "colorBrown", top_4['colorNo'], True)
top_4.insert(14, "colorPurple", top_4['colorNo'], True)
top_4.insert(14, "colorBlue", top_4['colorNo'], True)
top_4.insert(14, "colorGreen", top_4['colorNo'], True)
top_4.insert(14, "colorYellow", top_4['colorNo'], True)
top_4.insert(14, "colorIvory", top_4['colorNo'], True)
top_4.insert(14, "colorOrange", top_4['colorNo'], True)
top_4.insert(14, "colorPink", top_4['colorNo'], True)
top_4.insert(14, "colorRed", top_4['colorNo'], True)
top_4.insert(14, "colorBlack", top_4['colorNo'], True)
top_4.insert(14, "colorGrey", top_4['colorNo'], True)
top_4.insert(14, "colorWhite", top_4['colorNo'], True)

In [137]:
top_4['colorWhite'] = top_4['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_4['colorGrey'] = top_4['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_4['colorBlack'] = top_4['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_4['colorRed'] = top_4['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_4['colorPink'] = top_4['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_4['colorOrange'] = top_4['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_4['colorIvory'] = top_4['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_4['colorYellow'] = top_4['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_4['colorGreen'] = top_4['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_4['colorBlue'] = top_4['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_4['colorPurple'] = top_4['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_4['colorBrown'] = top_4['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_4['colorBeige'] = top_4['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_4['colorJean'] = top_4['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_4['colorPattern'] = top_4['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_4['colorOthers'] = top_4['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [138]:
print(top_4.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [139]:
top_4 = top_4.drop('colorNo', axis=1)

In [140]:
# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_4[top_4.columns[i]].value_counts())
print(top_4.shape)

0    34280
Name: colorWhite, dtype: int64
0    34280
Name: colorGrey, dtype: int64
0    34280
Name: colorBlack, dtype: int64
0    33783
1      497
Name: colorRed, dtype: int64
0    34280
Name: colorPink, dtype: int64
0    34280
Name: colorOrange, dtype: int64
0    34280
Name: colorIvory, dtype: int64
0    34280
Name: colorYellow, dtype: int64
0    34280
Name: colorGreen, dtype: int64
0    24120
1    10160
Name: colorBlue, dtype: int64
0    30943
1     3337
Name: colorPurple, dtype: int64
0    31335
1     2945
Name: colorBrown, dtype: int64
0    26571
1     7709
Name: colorBeige, dtype: int64
0    33746
1      534
Name: colorJean, dtype: int64
0    31352
1     2928
Name: colorPattern, dtype: int64
0    28110
1     6170
Name: colorOthers, dtype: int64
(34280, 48)


In [141]:
top_4.insert(29, "colorName", True)

In [142]:
top_4['colorName'] = top_4.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [143]:
top_4['colorName'].value_counts()

blue       10160
beige       7709
others      6170
purple      3337
brown       2945
pattern     2928
jean         534
red          497
Name: colorName, dtype: int64

### Price 전처리

In [144]:
top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace(",", ""))
top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace("원", ""))
top_4['clothPrice']

0         42000
1         65600
2         78400
3         44500
4        136500
          ...  
34275     58000
34276     66400
34277      8900
34278     69300
34279     29900
Name: clothPrice, Length: 34280, dtype: object

In [145]:
top_4['clothPrice'] = top_4['clothPrice'].astype('int')

In [146]:
top_4['clothReviewNo'] = top_4['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_4['clothReviewNo'] = top_4['clothReviewNo'].astype('int')
top_4['clothReviewNo']

0         0
1         5
2         0
3        14
4         0
         ..
34275     0
34276     0
34277     0
34278     0
34279     0
Name: clothReviewNo, Length: 34280, dtype: int32

In [147]:
top_4.index = top_4['clothId']
top_4 = top_4.drop('clothId', True)
top_4

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\1280914450.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  top_4 = top_4.drop('clothId', True)


,largeCategory,largeCategoryName,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
clothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1766962,1,상의,0,0,0,1,0,0,0,0,0,후드 티셔츠,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,blue,더블인텐션 03_NVCH,상상발전소,//image.msscdn.net/images/goods_img/20210128/1...,42000,20210128,"['후드티', '후디', '후드', '후드티셔츠', '남녀공용', '스마일스컬']",1,0,0.0,0,0,0,0,0,0,0,0,0,0
2221491,1,상의,0,0,0,0,0,1,0,0,0,니트/스웨터,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,blue,COLOR FRILL KNIT_NAVY,엽페,//image.msscdn.net/images/goods_img/20211108/2...,65600,20211108,[],0,1,5.0,5,0,0,0,0,0,0,0,0,0
2221502,1,상의,0,0,0,0,0,1,0,0,0,니트/스웨터,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,blue,CASHMERE STRIPE KNIT_NAVY,엽페,//image.msscdn.net/images/goods_img/20211108/2...,78400,20211108,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0
2273575,1,상의,0,0,0,0,0,1,0,0,0,니트/스웨터,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,blue,pnv031 캐시미어 울 뮤지엄 아트웍 니트 (네이비),파노브,//image.msscdn.net/images/goods_img/20211220/2...,44500,20211220,"['남자니트', '여자니트', '울니트', '라운드니트', '스웨터', '캐시미어']",1,0,5.0,14,0,0,0,0,0,0,0,0,0
2273811,1,상의,0,0,0,0,0,1,0,0,0,니트/스웨터,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,blue,BELL SLEEVE KNIT_NAVY,퍼블리카,//image.msscdn.net/images/goods_img/20211220/2...,136500,20211220,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413008,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,(163082198) 유니티 앤 리스펙트 반팔티 BLACK,오베이,//image.msscdn.net/images/goods_img/20200421/1...,58000,20200421,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
779965,1,상의,0,1,0,0,0,0,0,0,0,셔츠/블라우스,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,(W) Crown Crane Half Shirts - Red,아파트먼트,//image.msscdn.net/images/goods_img/20180514/7...,66400,20180514,"['반팔셔츠', '알로하셔츠', '여성셔츠']",0,1,0.0,0,3,3,3,3,1,0,1,0,0
1353489,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,[슈나이더] (남성)CDP 스판 라운드 티셔츠_SG2M1ST04,슈나이더,//image.msscdn.net/images/goods_img/20200316/1...,8900,20200316,"['라운드', '스판']",1,0,0.0,0,0,0,0,0,0,0,0,0,0


In [148]:
top_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34280 entries, 1766962 to 474858
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   largeCategory            34280 non-null  int64  
 1   largeCategoryName        34280 non-null  object 
 2   smallCategoryHalfshort   34280 non-null  int64  
 3   smallCategoryShirt       34280 non-null  int64  
 4   smallCategoryCollar      34280 non-null  int64  
 5   smallCategoryHoody       34280 non-null  int64  
 6   smallCategorySweatshirt  34280 non-null  int64  
 7   smallCategoryKnit        34280 non-null  int64  
 8   smallCategoryLong        34280 non-null  int64  
 9   smallCategoryShort       34280 non-null  int64  
 10  smallCategoryOthers      34280 non-null  int64  
 11  smallCategoryName        34280 non-null  object 
 12  colorWhite               34280 non-null  int64  
 13  colorGrey                34280 non-null  int64  
 14  colorBlack     

In [149]:
df_top = pd.concat([top_1, top_2, top_3, top_4], axis = 0)

In [150]:
df_top.shape

(157889, 48)

In [151]:
df_top.head()

,largeCategory,largeCategoryName,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
clothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1929966,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,화이트티셔츠(153),마리아쥬드비엔,//image.msscdn.net/images/goods_img/20210429/1...,59000,20210429,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
1927658,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,EMBLEM SLOGAN LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,43000,20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,0,0.0,0,0,0,0,0,0,0,0,0,0
1927660,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,CUTTING PATCH LOGO T-WHITE,오와이,//image.msscdn.net/images/goods_img/20210429/1...,43000,20210429,"['티셔츠', '반팔', '반팔티', '로고티셔츠', '반팔티셔츠', '그래픽티셔츠...",1,0,4.5,2,0,0,0,0,0,0,0,0,0
1928209,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,CLUB ARCH LOGO TEE (WHITE),스텝온리,//image.msscdn.net/images/goods_img/20210429/1...,26600,20210429,[],0,1,5.0,1,0,0,0,0,0,0,0,0,0
1927270,1,상의,1,0,0,0,0,0,0,0,0,반팔 티셔츠,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,white,Q221-3521-1WT 남성 라운드티셔츠,프로월드컵,//image.msscdn.net/images/goods_img/20210429/1...,24000,20210429,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0


In [152]:
df_top.to_csv('final_top.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')

# Outer

In [153]:
outer = pd.read_csv('./musinsa_clothes_outer.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\3755538683.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  outer = pd.read_csv('./musinsa_clothes_outer.csv')


In [154]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [155]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55039 entries, 0 to 55038
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            55039 non-null  int64  
 1   largeCategory      55039 non-null  int64  
 2   largeCategoryName  55039 non-null  object 
 3   smallCategory      55039 non-null  object 
 4   smallCategoryName  55039 non-null  object 
 5   colorNo            55039 non-null  int64  
 6   clothName          55039 non-null  object 
 7   brand              55039 non-null  object 
 8   image              55039 non-null  object 
 9   clothPrice         55039 non-null  object 
 10  date               55039 non-null  int64  
 11  hashtags           55039 non-null  object 
 12  clothSexMen        55039 non-null  int64  
 13  clothSexWomen      55039 non-null  int64  
 14  clothSexCommon     55039 non-null  int64  
 15  clothRate          55039 non-null  float64
 16  clothReviewNo      550

In [156]:
outer.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,5.503900e+04,55039.0,55039.000000,5.503900e+04,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.00000
mean,1.935946e+06,2.0,13.667290,2.020825e+07,0.585512,0.372118,0.042261,2.237203,1.057632,0.648177,0.734316,1.297444,0.965915,0.192409,0.045095,0.217864,0.17315
std,4.423802e+05,0.0,14.269903,1.106843e+04,0.492638,0.483374,0.201186,2.377868,1.794288,1.184583,1.389274,2.131908,1.641572,0.394196,0.207515,0.412798,0.37838
min,1.878100e+04,2.0,1.000000,2.012051e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.682430e+06,2.0,2.000000,2.020111e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2.117505e+06,2.0,5.000000,2.021091e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,2.259102e+06,2.0,24.000000,2.021121e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,1.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.00000
max,2.413481e+06,2.0,71.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,5.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.00000


In [157]:
outer = outer.drop_duplicates(subset = "clothId")
outer = outer.reset_index(drop=True)

In [158]:
outer['smallCategory'] = outer['smallCategory'].astype('str')

In [159]:
outer['largeCategory'].unique()

array([2], dtype=int64)

In [160]:
outer['largeCategoryName'].unique()

array(['아우터'], dtype=object)

In [161]:
outer['smallCategory'].unique()

array(['218', '220', '201', '212', '206', '221', '213', '207', '222',
       '215', '219', '223', '209', '217', '216', '203', '214', '225',
       '202', '224', '208', '204', '2s='], dtype=object)

In [162]:
outer['smallCategoryName'].unique()

array(['트레이닝 재킷', '카디건', '블루종/MA-1', '숏패딩/숏헤비 아우터', '나일론/코치  재킷', '베스트',
       '롱패딩/롱헤비 아우터', '겨울 싱글 코트', '후드 집업', '기타 아우터', '아노락 재킷', '플리스/뽀글이',
       '겨울 기타 코트', '트러커 재킷', '패딩 베스트', '슈트/블레이저 재킷', '사파리/헌팅 재킷', '무스탕/퍼',
       '레더/라이더스 재킷', '겨울 더블 코트', '환절기 코트', '스타디움 재킷',
       '\r\n            (휠라)        '], dtype=object)

In [163]:
outer[outer['smallCategoryName']=='\r\n            (휠라)        ']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
11366,1532828,2,아우터,2s=,\r\n (휠라),23,에센셜 리니어 보아 후드 자켓 (FE2JKD6101XCRM),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"79,000원",20200730,"['보아', '자켓', '플리스', '후드', '후리스', '캐리오버', '온라인전...",0,0,1,4.8,355,0,0,0,0,0,0,0,0,0
16581,1532833,2,아우터,2s=,\r\n (휠라),3,리니어 와펜 본딩 보아 자켓 (FE2JKC6102XGRY),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"69,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '온라인전용']",0,0,1,4.6,483,0,0,0,0,0,0,0,0,0
29633,1532832,2,아우터,2s=,\r\n (휠라),2,우븐 포켓 보아 자켓 (FE2JKD6103XBLK),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"89,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '캐리오버', '온라인전용']",0,1,0,4.6,483,0,0,0,0,0,0,0,0,0


In [164]:
outer.loc[outer.smallCategoryName=='\r\n            (휠라)        ', ('smallCategoryName')] = '기타 아우터'

In [165]:
outer[outer['smallCategory']=="2s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
11366,1532828,2,아우터,2s=,기타 아우터,23,에센셜 리니어 보아 후드 자켓 (FE2JKD6101XCRM),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"79,000원",20200730,"['보아', '자켓', '플리스', '후드', '후리스', '캐리오버', '온라인전...",0,0,1,4.8,355,0,0,0,0,0,0,0,0,0
16581,1532833,2,아우터,2s=,기타 아우터,3,리니어 와펜 본딩 보아 자켓 (FE2JKC6102XGRY),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"69,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '온라인전용']",0,0,1,4.6,483,0,0,0,0,0,0,0,0,0
29633,1532832,2,아우터,2s=,기타 아우터,2,우븐 포켓 보아 자켓 (FE2JKD6103XBLK),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"89,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '캐리오버', '온라인전용']",0,1,0,4.6,483,0,0,0,0,0,0,0,0,0


In [166]:
outer.loc[outer.smallCategory=='2s=', ('smallCategory')] = '215'

In [167]:
outer['smallCategory'] = outer['smallCategory'].astype('int')

In [168]:
outer['smallCategory'].value_counts()

220    6148
203    5378
212    4926
206    4592
222    4118
215    4059
223    3310
221    3178
201    2253
217    2172
208    2057
202    1732
219    1587
218    1576
214    1457
207    1400
209     946
216     912
225     879
213     836
224     798
204     649
Name: smallCategory, dtype: int64

### smallCategory 원핫인코딩 변환

In [169]:
outer.insert(4, "smallCategoryOthers", outer['smallCategory'], True)
outer.insert(4, "smallCategoryJacket", outer['smallCategory'], True)
outer.insert(4, "smallCategoryVest", outer['smallCategory'], True)
outer.insert(4, "smallCategoryPaddedcoat", outer['smallCategory'], True)
outer.insert(4, "smallCategoryCoatt", outer['smallCategory'], True)
outer.insert(4, "smallCategoryFleece", outer['smallCategory'], True)
outer.insert(4, "smallCategoryCardigan", outer['smallCategory'], True)
outer.insert(4, "smallCategoryMustang", outer['smallCategory'], True)
outer.insert(4, "smallCategoryRiders", outer['smallCategory'], True)
outer.insert(4, "smallCategoryBlouson", outer['smallCategory'], True)
outer.insert(4, "smallCategoryHoodie", outer['smallCategory'], True)

In [170]:
outer['smallCategoryHoodie'] = outer['smallCategoryHoodie'].apply(lambda x: 1 if x == 222 else 0)
outer['smallCategoryBlouson'] = outer['smallCategoryBlouson'].apply(lambda x: 1 if x == 201 else 0)
outer['smallCategoryRiders'] = outer['smallCategoryRiders'].apply(lambda x: 1 if x == 202 else 0)
outer['smallCategoryMustang'] = outer['smallCategoryMustang'].apply(lambda x: 1 if x == 225 else 0)
outer['smallCategoryCardigan'] = outer['smallCategoryCardigan'].apply(lambda x: 1 if x == 220 else 0)
outer['smallCategoryFleece'] = outer['smallCategoryFleece'].apply(lambda x: 1 if x == 223 else 0)
outer['smallCategoryCoatt'] = outer['smallCategoryCoatt'].apply(lambda x: 1 if x in [208, 207, 224, 209] else 0)
outer['smallCategoryPaddedcoat'] = outer['smallCategoryPaddedcoat'].apply(lambda x: 1 if x in [212, 213, 216] else 0)
outer['smallCategoryVest'] = outer['smallCategoryVest'].apply(lambda x: 1 if x == 221 else 0)
outer['smallCategoryJacket'] = outer['smallCategoryJacket'].apply(lambda x: 1 if x in [217, 203, 219, 218, 204, 214, 206] else 0)
outer['smallCategoryOthers'] = outer['smallCategoryOthers'].apply(lambda x: 1 if x == 215 else 0)

In [171]:
outer = outer.drop('smallCategory', axis=1)

In [172]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoatt',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorNo', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [173]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 14):
    num += outer[outer.columns[i]].value_counts()[1]
    print(outer[outer.columns[i]].value_counts())
print(num)
print(outer.shape)

0    50845
1     4118
Name: smallCategoryHoodie, dtype: int64
0    52710
1     2253
Name: smallCategoryBlouson, dtype: int64
0    53231
1     1732
Name: smallCategoryRiders, dtype: int64
0    54084
1      879
Name: smallCategoryMustang, dtype: int64
0    48815
1     6148
Name: smallCategoryCardigan, dtype: int64
0    51653
1     3310
Name: smallCategoryFleece, dtype: int64
0    49762
1     5201
Name: smallCategoryCoatt, dtype: int64
0    48289
1     6674
Name: smallCategoryPaddedcoat, dtype: int64
0    51785
1     3178
Name: smallCategoryVest, dtype: int64
0    37552
1    17411
Name: smallCategoryJacket, dtype: int64
0    50904
1     4059
Name: smallCategoryOthers, dtype: int64
54963
(54963, 36)


### clothSexCommon 변환

In [174]:
outer['clothSexMen'] = outer.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
outer['clothSexWomen'] = outer.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [175]:
outer = outer.drop('clothSexCommon', axis=1)

In [176]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoatt',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorNo', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [177]:
outer['clothSexMen'] = outer.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
outer['clothSexWomen'] = outer.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [178]:
print(outer['clothSexMen'].value_counts())
print(outer['clothSexWomen'].value_counts())
print(outer.shape)

1    34493
0    20470
Name: clothSexMen, dtype: int64
0    32168
1    22795
Name: clothSexWomen, dtype: int64
(54963, 35)


### color 원핫인코딩 변환

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [179]:
outer.insert(15, "colorOthers", outer['colorNo'], True)
outer.insert(15, "colorPattern", outer['colorNo'], True)
outer.insert(15, "colorJean", outer['colorNo'], True)
outer.insert(15, "colorBeige", outer['colorNo'], True)
outer.insert(15, "colorBrown", outer['colorNo'], True)
outer.insert(15, "colorPurple", outer['colorNo'], True)
outer.insert(15, "colorBlue", outer['colorNo'], True)
outer.insert(15, "colorGreen", outer['colorNo'], True)
outer.insert(15, "colorYellow", outer['colorNo'], True)
outer.insert(15, "colorIvory", outer['colorNo'], True)
outer.insert(15, "colorOrange", outer['colorNo'], True)
outer.insert(15, "colorPink", outer['colorNo'], True)
outer.insert(15, "colorRed", outer['colorNo'], True)
outer.insert(15, "colorBlack", outer['colorNo'], True)
outer.insert(15, "colorGrey", outer['colorNo'], True)
outer.insert(15, "colorWhite", outer['colorNo'], True)

In [180]:
outer['colorWhite'] = outer['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
outer['colorGrey'] = outer['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
outer['colorBlack'] = outer['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
outer['colorRed'] = outer['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
outer['colorPink'] = outer['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
outer['colorOrange'] = outer['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
outer['colorIvory'] = outer['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
outer['colorYellow'] = outer['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
outer['colorGreen'] = outer['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
outer['colorBlue'] = outer['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
outer['colorPurple'] = outer['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
outer['colorBrown'] = outer['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
outer['colorBeige'] = outer['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
outer['colorJean'] = outer['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
outer['colorPattern'] = outer['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
outer['colorOthers'] = outer['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [181]:
outer = outer.drop('colorNo', axis=1)

In [182]:
print(outer.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoatt',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorWhite', 'colorGrey',
       'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory',
       'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown',
       'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [183]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(15, 31):
    num+=outer[outer.columns[i]].value_counts()[1]
    print(outer[outer.columns[i]].value_counts())
print(num)
print(outer.shape)

0    53068
1     1895
Name: colorWhite, dtype: int64
0    48564
1     6399
Name: colorGrey, dtype: int64
0    38979
1    15984
Name: colorBlack, dtype: int64
0    54066
1      897
Name: colorRed, dtype: int64
0    53953
1     1010
Name: colorPink, dtype: int64
0    54538
1      425
Name: colorOrange, dtype: int64
0    51706
1     3257
Name: colorIvory, dtype: int64
0    54235
1      728
Name: colorYellow, dtype: int64
0    50135
1     4828
Name: colorGreen, dtype: int64
0    48743
1     6220
Name: colorBlue, dtype: int64
0    54257
1      706
Name: colorPurple, dtype: int64
0    52678
1     2285
Name: colorBrown, dtype: int64
0    48963
1     6000
Name: colorBeige, dtype: int64
0    54186
1      777
Name: colorJean, dtype: int64
0    54072
1      891
Name: colorPattern, dtype: int64
0    52302
1     2661
Name: colorOthers, dtype: int64
54963
(54963, 50)


In [184]:
outer.insert(31, "colorName", True)

In [185]:
outer['colorName'] = outer.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [186]:
outer['colorName'].value_counts()

black      15984
grey        6399
blue        6220
beige       6000
green       4828
ivory       3257
others      2661
brown       2285
white       1895
pink        1010
red          897
pattern      891
jean         777
yellow       728
purple       706
orange       425
Name: colorName, dtype: int64

### Price 전처리

In [187]:
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace(",", ""))
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace("원", ""))
outer['clothPrice']

0        109000
1         38700
2        159000
3        210000
4         95400
          ...  
54958    118300
54959    152100
54960    169200
54961    153000
54962    286000
Name: clothPrice, Length: 54963, dtype: object

In [188]:
outer['clothPrice'] = outer['clothPrice'].astype('int')

In [189]:
outer['clothReviewNo'] = outer['clothReviewNo'].apply(lambda x: x.replace(",", ""))
outer['clothReviewNo'] = outer['clothReviewNo'].astype('int')
outer['clothReviewNo']

0          0
1         16
2          0
3          0
4          0
        ... 
54958    424
54959    279
54960     37
54961      0
54962      0
Name: clothReviewNo, Length: 54963, dtype: int32

In [190]:
outer.index = outer['clothId']
outer = outer.drop('clothId', True)
outer

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\2618014124.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  outer = outer.drop('clothId', True)


,largeCategory,largeCategoryName,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoatt,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
clothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2401813,2,아우터,0,0,0,0,0,0,0,0,0,1,0,트레이닝 재킷,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,[PWX]Q122-2206-1ME 여성 2.5LAYER 자켓,프로월드컵,//image.msscdn.net/images/goods_img/20220307/2...,109000,20220307,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0
2340052,2,아우터,0,0,0,0,1,0,0,0,0,0,0,카디건,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,[NF] 믹스 버튼 가디건 (DEEP PINK)_F22QB401,네스티팬시클럽,//image.msscdn.net/images/goods_img/20220204/2...,38700,20220204,"['크롭가디건', '가디건', '데일리가디건', '여자가디건', '꾸안꾸', '봄아...",0,1,4.9,16,2,3,3,5,3,0,0,0,0
2379678,2,아우터,0,1,0,0,0,0,0,0,0,0,0,블루종/MA-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,N222MJP901 시티 캐주얼 립소재 세미오버핏 MA-1 항공 점퍼 DUSTY ROSE,내셔널지오그래픽,//image.msscdn.net/images/goods_img/20220223/2...,159000,20220223,"['22S/S컬렉션', '자켓', '봄아우터', '바람막이', '아우터', '봄자켓...",1,0,0.0,0,0,0,0,0,0,0,0,0,0
2360803,2,아우터,0,0,0,0,0,0,0,1,0,0,0,숏패딩/숏헤비 아우터,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,[FW19] 넥시스 재킷 여성,아크테릭스,//image.msscdn.net/images/goods_img/20220215/2...,210000,20220215,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0
2354237,2,아우터,0,0,0,0,0,0,0,0,0,1,0,나일론/코치 재킷,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,우먼스 루즈핏 스트링 자켓 (UM122CJK21),엄브로,//image.msscdn.net/images/goods_img/20220211/2...,95400,20220211,"['WOMENS루즈핏스트링자켓', '스트링', '얼리버드쿠폰']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266813,2,아우터,0,0,1,0,0,0,0,0,0,0,0,레더/라이더스 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,SINGLE RIDERS JACKET JS [BLACK],비바스튜디오,//image.msscdn.net/images/goods_img/20151021/2...,118300,20151021,"['가죽자켓', '라이더', '라이더자켓', '라이더재킷', '레더컬렉션', '패밀...",1,0,4.5,424,0,0,0,0,0,0,0,0,0
2080918,2,아우터,0,0,0,0,0,0,0,0,0,1,0,나일론/코치 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,윈드브레이커 자켓 (블랙),쿠어,//image.msscdn.net/images/goods_img/20210823/2...,152100,20210823,"['쿠어자켓', '자켓', '윈드브레이커', '바람막이', '간절기아우터', '21...",1,1,4.9,279,0,0,0,0,0,0,0,0,0
2322135,2,아우터,0,0,0,0,0,0,0,0,0,1,0,스타디움 재킷,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,black,딜런 스카잔 바시티 재킷 블랙,엘무드,//image.msscdn.net/images/goods_img/20220124/2...,169200,20220124,"['22S/S2차발매', '22S/S컬렉션', '바시티', '스카잔', '세미오버'...",1,0,4.8,37,0,0,0,0,0,0,0,0,0


In [191]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54963 entries, 2401813 to 2398671
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   largeCategory            54963 non-null  int64  
 1   largeCategoryName        54963 non-null  object 
 2   smallCategoryHoodie      54963 non-null  int64  
 3   smallCategoryBlouson     54963 non-null  int64  
 4   smallCategoryRiders      54963 non-null  int64  
 5   smallCategoryMustang     54963 non-null  int64  
 6   smallCategoryCardigan    54963 non-null  int64  
 7   smallCategoryFleece      54963 non-null  int64  
 8   smallCategoryCoatt       54963 non-null  int64  
 9   smallCategoryPaddedcoat  54963 non-null  int64  
 10  smallCategoryVest        54963 non-null  int64  
 11  smallCategoryJacket      54963 non-null  int64  
 12  smallCategoryOthers      54963 non-null  int64  
 13  smallCategoryName        54963 non-null  object 
 14  colorWhite    

In [192]:
outer.to_csv('final_outer.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')

# Pants

In [193]:
pants = pd.read_csv('./musinsa_clothes_pants.csv')

In [194]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [195]:
pants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57679 entries, 0 to 57678
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            57679 non-null  int64  
 1   largeCategory      57679 non-null  int64  
 2   largeCategoryName  57679 non-null  object 
 3   smallCategory      57679 non-null  int64  
 4   smallCategoryName  57679 non-null  object 
 5   colorNo            57679 non-null  int64  
 6   clothName          57679 non-null  object 
 7   brand              57679 non-null  object 
 8   image              57679 non-null  object 
 9   clothPrice         57679 non-null  object 
 10  date               57679 non-null  int64  
 11  hashtags           57679 non-null  object 
 12  clothSexMen        57679 non-null  int64  
 13  clothSexWomen      57679 non-null  int64  
 14  clothSexCommon     57679 non-null  int64  
 15  clothRate          57679 non-null  float64
 16  clothReviewNo      576

In [196]:
pants.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,5.767900e+04,57679.0,57679.000000,57679.000000,5.767900e+04,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000
mean,1.840605e+06,3.0,305.918601,15.812115,2.020691e+07,0.645868,0.324295,0.029768,2.475041,1.118345,0.775482,0.989962,1.485844,0.960037,0.248669,0.183620,0.251443,0.175038
std,4.652932e+05,0.0,2.625236,16.485140,1.145007e+04,0.478254,0.468115,0.169949,2.367970,1.739613,1.266744,1.657343,2.221116,1.478658,0.432245,0.387177,0.433847,0.380003
min,2.586800e+04,3.0,302.000000,1.000000,2.012110e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.564060e+06,3.0,304.000000,2.000000,2.020083e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.945703e+06,3.0,306.000000,7.000000,2.021051e+07,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.204482e+06,3.0,308.000000,25.000000,2.021103e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,2.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000
max,2.413615e+06,3.0,310.000000,71.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [197]:
pants = pants.drop_duplicates(subset = "clothId")
pants = pants.reset_index(drop=True)

In [198]:
pants['smallCategory'] = pants['smallCategory'].astype('str')

In [199]:
pants['largeCategory'].unique()

array([3], dtype=int64)

In [200]:
pants['largeCategoryName'].unique()

array(['바지'], dtype=object)

In [201]:
pants['smallCategory'].unique()

array(['306', '304', '308', '309', '302', '310', '307', '305'],
      dtype=object)

In [202]:
pants['smallCategoryName'].unique()

array(['기타 바지', '트레이닝/조거 팬츠', '슈트 팬츠/슬랙스', '숏 팬츠', '데님 팬츠', '기능성 하의',
       '점프 슈트/오버올', '코튼 팬츠', '레깅스'], dtype=object)

In [203]:
pants[pants['smallCategoryName']=='기능성 하의']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
32,1321523,3,바지,310,기능성 하의,3,여성 요가복 DEVI-B0036-그레이 필라테스 레깅스 9부 베이직,데비웨어,//image.msscdn.net/images/goods_img/20200225/1...,"24,000원",20200225,"['여성요가복', '요가복', '필라테스', '필라테스복', '필라테스운동복', '...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
34,1667376,3,바지,310,기능성 하의,3,여성 사이드 스타프린트 레깅스 GREY / FF0WLG01F87,피피라핀,//image.msscdn.net/images/goods_img/20201028/1...,"29,900원",20201028,"['레깅스', '여성레깅스', '겨울레깅스', '여자레깅스', '케주얼레깅스']",0,1,0,4.0,2,1,3,5,5,3,1,0,1,1
138,2316809,3,바지,310,기능성 하의,3,기모_CREORA & CELLA LEGGINS GR,아이코노그라피,//image.msscdn.net/images/goods_img/20220120/2...,"19,800원",20220120,"['레깅스추천', '레깅스', '여성레깅스', '레깅스컬렉션', '레깅스팬츠', '...",0,1,0,0.0,0,1,1,5,5,3,1,0,1,1
146,2185722,3,바지,310,기능성 하의,3,RIF (리프) 여성 본딩 타이즈 팬츠_Ciment,아이더,//image.msscdn.net/images/goods_img/20211019/2...,"129,000원",20211019,[],0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
161,2301479,3,바지,310,기능성 하의,3,TS 럭스 하이라이즈 레깅스 - 그레이 / HB2028,리복,//image.msscdn.net/images/goods_img/20220111/2...,"44,900원",20220111,"['타이츠', '하이웨스트레깅스', '하이웨스트타이츠', '요가', '필라테스', ...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57567,1161106,3,바지,310,기능성 하의,15,여성 사티 팬츠(하의) 4종 시리즈,69슬램,//image.msscdn.net/images/goods_img/20190920/1...,"69,000원",20190920,"['래쉬팬츠', '레깅스', '엑티브웨어', '여성요가복하의', '여성필라테스하의']",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57579,876594,3,바지,310,기능성 하의,15,액티브웨어 레깅스 센느 네이비,밸롭,//image.msscdn.net/images/goods_img/20181010/8...,"39,000원",20181010,[],0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57583,1614139,3,바지,310,기능성 하의,15,CLWP7004 7부 팬츠 마젠타,씨엘코코,//image.msscdn.net/images/goods_img/20200921/1...,"13,900원",20200921,"['요가복', '필라테스복', '레깅스', '요가복상의', '요가복하의', '요가복...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57592,1441580,3,바지,310,기능성 하의,15,AW01PL04 아이스 숏 레깅스_다크 그레이,애스플래폼,//image.msscdn.net/images/goods_img/20200512/1...,"39,000원",20200512,"['그레이레깅스', '레깅스반바지', '반바지레깅스', '숏레깅스', '아이스레깅스...",0,1,0,0.0,0,1,1,5,2,2,0,1,0,0


In [204]:
pants.loc[pants.smallCategoryName=='기능성 하의', ('smallCategoryName')] = '레깅스'
pants.loc[pants.smallCategoryName=='기능성 하의', ('smallCategory')] = '305'

In [205]:
pants['smallCategory'] = pants['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [206]:
pants.insert(4, "smallCategoryOthers", pants['smallCategory'], True)
pants.insert(4, "smallCategoryJumpsuit", pants['smallCategory'], True)
pants.insert(4, "smallCategoryLeggings", pants['smallCategory'], True)
pants.insert(4, "smallCategoryShortpants", pants['smallCategory'], True)
pants.insert(4, "smallCategoryJoggerpants", pants['smallCategory'], True)
pants.insert(4, "smallCategorySlacks", pants['smallCategory'], True)
pants.insert(4, "smallCategoryCottonpants", pants['smallCategory'], True)
pants.insert(4, "smallCategoryDenimpants", pants['smallCategory'], True)

In [207]:
pants['smallCategoryDenimpants'] = pants['smallCategoryDenimpants'].apply(lambda x: 1 if x == 302 else 0)
pants['smallCategoryCottonpants'] = pants['smallCategoryCottonpants'].apply(lambda x: 1 if x == 307 else 0)
pants['smallCategorySlacks'] = pants['smallCategorySlacks'].apply(lambda x: 1 if x == 308 else 0)
pants['smallCategoryJoggerpants'] = pants['smallCategoryJoggerpants'].apply(lambda x: 1 if x == 304 else 0)
pants['smallCategoryShortpants'] = pants['smallCategoryShortpants'].apply(lambda x: 1 if x == 309 else 0)
pants['smallCategoryLeggings'] = pants['smallCategoryLeggings'].apply(lambda x: 1 if x == 305 else 0)
pants['smallCategoryJumpsuit'] = pants['smallCategoryJumpsuit'].apply(lambda x: 1 if x == 310 else 0)
pants['smallCategoryOthers'] = pants['smallCategoryOthers'].apply(lambda x: 1 if x == 306 else 0)

In [208]:
pants = pants.drop('smallCategory', axis=1)

In [209]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [210]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 11):
    num += pants[pants.columns[i]].value_counts()[1]
    print(pants[pants.columns[i]].value_counts())
print(num)
print(pants.shape)

0    47646
1     9954
Name: smallCategoryDenimpants, dtype: int64
0    49928
1     7672
Name: smallCategoryCottonpants, dtype: int64
0    50396
1     7204
Name: smallCategorySlacks, dtype: int64
0    42486
1    15114
Name: smallCategoryJoggerpants, dtype: int64
0    47026
1    10574
Name: smallCategoryShortpants, dtype: int64
0    57578
1       22
Name: smallCategoryLeggings, dtype: int64
0    54730
1     2870
Name: smallCategoryJumpsuit, dtype: int64
0    53410
1     4190
Name: smallCategoryOthers, dtype: int64
57600
(57600, 33)


### clothSexCommon 변환

In [211]:
pants['clothSexMen'] = pants.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
pants['clothSexWomen'] = pants.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [212]:
pants = pants.drop('clothSexCommon', axis=1)

In [213]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [214]:
pants['clothSexMen'] = pants.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
pants['clothSexWomen'] = pants.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [215]:
print(pants['clothSexMen'].value_counts())
print(pants['clothSexWomen'].value_counts())
print(pants.shape)

1    38914
0    18686
Name: clothSexMen, dtype: int64
0    37198
1    20402
Name: clothSexWomen, dtype: int64
(57600, 32)


### color 원핫인코딩 변환

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [216]:
pants.insert(13, "colorOthers", pants['colorNo'], True)
pants.insert(13, "colorPattern", pants['colorNo'], True)
pants.insert(13, "colorJean", pants['colorNo'], True)
pants.insert(13, "colorBeige", pants['colorNo'], True)
pants.insert(13, "colorBrown", pants['colorNo'], True)
pants.insert(13, "colorPurple", pants['colorNo'], True)
pants.insert(13, "colorBlue", pants['colorNo'], True)
pants.insert(13, "colorGreen", pants['colorNo'], True)
pants.insert(13, "colorYellow", pants['colorNo'], True)
pants.insert(13, "colorIvory", pants['colorNo'], True)
pants.insert(13, "colorOrange", pants['colorNo'], True)
pants.insert(13, "colorPink", pants['colorNo'], True)
pants.insert(13, "colorRed", pants['colorNo'], True)
pants.insert(13, "colorBlack", pants['colorNo'], True)
pants.insert(13, "colorGrey", pants['colorNo'], True)
pants.insert(13, "colorWhite", pants['colorNo'], True)

In [217]:
pants['colorWhite'] = pants['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
pants['colorGrey'] = pants['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
pants['colorBlack'] = pants['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
pants['colorRed'] = pants['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
pants['colorPink'] = pants['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
pants['colorOrange'] = pants['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
pants['colorIvory'] = pants['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
pants['colorYellow'] = pants['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
pants['colorGreen'] = pants['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
pants['colorBlue'] = pants['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
pants['colorPurple'] = pants['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
pants['colorBrown'] = pants['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
pants['colorBeige'] = pants['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
pants['colorJean'] = pants['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
pants['colorPattern'] = pants['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 69, 66] else 0)
pants['colorOthers'] = pants['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [218]:
print(pants.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed',
       'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen',
       'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean',
       'colorPattern', 'colorOthers', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')


In [219]:
pants = pants.drop('colorNo', axis=1)

In [220]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(12, 28):
    num += pants[pants.columns[i]].value_counts()[1]
    print(pants[pants.columns[i]].value_counts())
print(num)
print(pants.shape)

0    55827
1     1773
Name: colorWhite, dtype: int64
0    49481
1     8119
Name: colorGrey, dtype: int64
0    42691
1    14909
Name: colorBlack, dtype: int64
0    56943
1      657
Name: colorRed, dtype: int64
0    56783
1      817
Name: colorPink, dtype: int64
0    57163
1      437
Name: colorOrange, dtype: int64
0    54887
1     2713
Name: colorIvory, dtype: int64
0    57034
1      566
Name: colorYellow, dtype: int64
0    53309
1     4291
Name: colorGreen, dtype: int64
0    51080
1     6520
Name: colorBlue, dtype: int64
0    56953
1      647
Name: colorPurple, dtype: int64
0    55964
1     1636
Name: colorBrown, dtype: int64
0    52433
1     5167
Name: colorBeige, dtype: int64
0    51267
1     6333
Name: colorJean, dtype: int64
0    57187
1      413
Name: colorPattern, dtype: int64
0    54998
1     2602
Name: colorOthers, dtype: int64
57600
(57600, 47)


In [221]:
pants.insert(28, "colorName", True)

In [222]:
pants['colorName'] = pants.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [223]:
pants['colorName'].value_counts()

black      14909
grey        8119
blue        6520
jean        6333
beige       5167
green       4291
ivory       2713
others      2602
white       1773
brown       1636
pink         817
red          657
purple       647
yellow       566
orange       437
pattern      413
Name: colorName, dtype: int64

### Price 전처리

In [224]:
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace(",", ""))
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace("원", ""))
pants['clothPrice']

0        130000
1         69000
2        199000
3         95000
4         69000
          ...  
57595    149000
57596     70500
57597     69000
57598    110000
57599    171000
Name: clothPrice, Length: 57600, dtype: object

In [225]:
pants['clothPrice'] = pants['clothPrice'].astype('int')

In [226]:
pants['clothReviewNo'] = pants['clothReviewNo'].apply(lambda x: x.replace(",", ""))
pants['clothReviewNo'] = pants['clothReviewNo'].astype('int')
pants['clothReviewNo']

0        0
1        0
2        0
3        0
4        0
        ..
57595    0
57596    0
57597    0
57598    0
57599    0
Name: clothReviewNo, Length: 57600, dtype: int32

In [227]:
pants.index = pants['clothId']
pants = pants.drop('clothId', True)
pants

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\381069870.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  pants = pants.drop('clothId', True)


,largeCategory,largeCategoryName,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
clothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1971752,3,바지,0,0,0,0,0,0,0,1,기타 바지,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,카치아 61360203000 CACCIA 006 여자 울 팬츠,막스마라,//image.msscdn.net/images/goods_img/20210527/1...,130000,20210527,"['럭스보이', '위즈컴퍼니']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
726566,3,바지,0,0,0,1,0,0,0,0,트레이닝/조거 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,SSC TEAM TRACK PANTS (GREY),피피피,//image.msscdn.net/images/goods_img/20180228/7...,69000,20180228,"['나일론팬츠', '트랙팬츠', '트레이닝세트', '트레이닝팬츠', '팀복', '피...",1,0,0.0,0,0,0,0,0,0,0,0,0,0
2213618,3,바지,0,0,0,0,0,0,0,1,기타 바지,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,미니투어맨 패턴 팬츠_L213PT403P,피지에이투어,//image.msscdn.net/images/goods_img/20211103/2...,199000,20211103,"['골프바지', '골프팬츠', '스윙바지', '스윙팬츠', '필드바지', '필드팬츠...",1,0,0.0,0,0,0,0,0,0,0,0,0,0
2265443,3,바지,0,0,1,0,0,0,0,0,슈트 팬츠/슬랙스,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,[UNISEX] 턱 와이드 팬츠 그레이,니커보커,//image.msscdn.net/images/goods_img/20211210/2...,95000,20211210,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0
2005302,3,바지,0,0,0,0,1,0,0,0,숏 팬츠,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,grey,버뮤다 이지 숏 팬츠 OUOSP002-GY,오브젝트 뮤지엄,//image.msscdn.net/images/goods_img/20210622/2...,69000,20210622,"['반바지', '숏팬츠', '린넨', '코튼', '하프팬츠']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889420,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,남)프리미엄 스모킹 포인트 빈티지 워싱 AL1DTA15-DB,잠뱅이,//image.msscdn.net/images/goods_img/20210408/1...,149000,20210408,[],1,0,0.0,0,0,0,0,0,0,0,0,0,0
1275944,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,스키니스트레치지퍼데님팬츠 KJE8904_LB,코너스,//image.msscdn.net/images/goods_img/20200120/1...,70500,20200120,"['데님팬츠', '코너스데님팬츠']",1,0,0.0,0,0,0,0,0,0,0,0,0,0
1889504,3,바지,1,0,0,0,0,0,0,0,데님 팬츠,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,여)오비안밴드 바이오스톤 탈색워싱 AL1DRS61-LB,잠뱅이,//image.msscdn.net/images/goods_img/20210408/1...,69000,20210408,[],0,1,0.0,0,0,0,0,0,0,0,0,0,0


In [228]:
pants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57600 entries, 1971752 to 655497
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   largeCategory             57600 non-null  int64  
 1   largeCategoryName         57600 non-null  object 
 2   smallCategoryDenimpants   57600 non-null  int64  
 3   smallCategoryCottonpants  57600 non-null  int64  
 4   smallCategorySlacks       57600 non-null  int64  
 5   smallCategoryJoggerpants  57600 non-null  int64  
 6   smallCategoryShortpants   57600 non-null  int64  
 7   smallCategoryLeggings     57600 non-null  int64  
 8   smallCategoryJumpsuit     57600 non-null  int64  
 9   smallCategoryOthers       57600 non-null  int64  
 10  smallCategoryName         57600 non-null  object 
 11  colorWhite                57600 non-null  int64  
 12  colorGrey                 57600 non-null  int64  
 13  colorBlack                57600 non-null  int64  
 14 

In [229]:
pants.to_csv('final_pants.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')

# Onepiece

In [230]:
onepiece = pd.read_csv('./musinsa_clothes_onepiece.csv')

In [231]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [232]:
onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9456 entries, 0 to 9455
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            9456 non-null   int64  
 1   largeCategory      9456 non-null   int64  
 2   largeCategoryName  9456 non-null   object 
 3   smallCategory      9456 non-null   int64  
 4   smallCategoryName  9456 non-null   object 
 5   colorNo            9456 non-null   int64  
 6   clothName          9456 non-null   object 
 7   brand              9456 non-null   object 
 8   image              9456 non-null   object 
 9   clothPrice         9456 non-null   object 
 10  date               9456 non-null   int64  
 11  hashtags           9456 non-null   object 
 12  clothSexMen        9456 non-null   int64  
 13  clothSexWomen      9456 non-null   int64  
 14  clothSexCommon     9456 non-null   int64  
 15  clothRate          9456 non-null   float64
 16  clothReviewNo      9456 

In [233]:
onepiece.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,9.456000e+03,9456.0,9456.000000,9456.000000,9.456000e+03,9456.0,9456.0,9456.0,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000
mean,1.828642e+06,4.0,407.105859,16.600465,2.020675e+07,0.0,1.0,0.0,1.104209,1.119924,0.748519,0.949979,1.498731,0.940567,0.240376,0.202728,0.218591,0.118655
std,4.241448e+05,0.0,0.776100,17.107649,9.750778e+03,0.0,0.0,0.0,1.993787,1.610713,1.159970,1.539927,2.121980,1.386594,0.427335,0.402053,0.413313,0.323399
min,1.374480e+05,4.0,406.000000,1.000000,2.014103e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.509934e+06,4.0,406.000000,2.000000,2.020071e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.942870e+06,4.0,407.000000,8.000000,2.021051e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.143857e+06,4.0,408.000000,30.000000,2.021092e+07,0.0,1.0,0.0,0.000000,3.000000,1.000000,1.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,2.413328e+06,4.0,408.000000,71.000000,2.022031e+07,0.0,1.0,0.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [234]:
onepiece = onepiece.drop_duplicates(subset = "clothId")
onepiece = onepiece.reset_index(drop=True)

In [235]:
onepiece['smallCategory'] = onepiece['smallCategory'].astype('str')

In [236]:
onepiece['largeCategory'].unique()

array([4], dtype=int64)

In [237]:
onepiece['largeCategoryName'].unique()

array(['원피스'], dtype=object)

In [238]:
onepiece['smallCategory'].unique()

array(['407', '408', '406'], dtype=object)

In [239]:
onepiece['smallCategoryName'].unique()

array(['미디 원피스', '맥시 원피스', '미니 원피스'], dtype=object)

In [240]:
onepiece['smallCategory'] = onepiece['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [241]:
onepiece.insert(4, "smallCategoryMaxidress", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategoryMidi", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategoryMinidress", onepiece['smallCategory'], True)

In [242]:
onepiece['smallCategoryMinidress'] = onepiece['smallCategoryMinidress'].apply(lambda x: 1 if x == 406 else 0)
onepiece['smallCategoryMidi'] = onepiece['smallCategoryMidi'].apply(lambda x: 1 if x == 407 else 0)
onepiece['smallCategoryMaxidress'] = onepiece['smallCategoryMaxidress'].apply(lambda x: 1 if x == 408 else 0)

In [243]:
onepiece = onepiece.drop('smallCategory', axis=1)

In [244]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [245]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 6):
    num += onepiece[onepiece.columns[i]].value_counts()[1]
    print(onepiece[onepiece.columns[i]].value_counts())
print(num)
print(onepiece.shape)

0    7054
1    2394
Name: smallCategoryMinidress, dtype: int64
0    5793
1    3655
Name: smallCategoryMidi, dtype: int64
0    6049
1    3399
Name: smallCategoryMaxidress, dtype: int64
9448
(9448, 28)


### clothSexCommon 변환

In [246]:
onepiece['clothSexMen'] = onepiece.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
onepiece['clothSexWomen'] = onepiece.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [247]:
onepiece = onepiece.drop('clothSexCommon', axis=1)

In [248]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [249]:
onepiece['clothSexMen'] = onepiece.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
onepiece['clothSexWomen'] = onepiece.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [250]:
print(onepiece['clothSexMen'].value_counts())
print(onepiece['clothSexWomen'].value_counts())
print(onepiece.shape)

0    9448
Name: clothSexMen, dtype: int64
1    9448
Name: clothSexWomen, dtype: int64
(9448, 27)


### color 원핫인코딩 변환

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [251]:
onepiece.insert(8, "colorOthers", onepiece['colorNo'], True)
onepiece.insert(8, "colorPattern", onepiece['colorNo'], True)
onepiece.insert(8, "colorJean", onepiece['colorNo'], True)
onepiece.insert(8, "colorBeige", onepiece['colorNo'], True)
onepiece.insert(8, "colorBrown", onepiece['colorNo'], True)
onepiece.insert(8, "colorPurple", onepiece['colorNo'], True)
onepiece.insert(8, "colorBlue", onepiece['colorNo'], True)
onepiece.insert(8, "colorGreen", onepiece['colorNo'], True)
onepiece.insert(8, "colorYellow", onepiece['colorNo'], True)
onepiece.insert(8, "colorIvory", onepiece['colorNo'], True)
onepiece.insert(8, "colorOrange", onepiece['colorNo'], True)
onepiece.insert(8, "colorPink", onepiece['colorNo'], True)
onepiece.insert(8, "colorRed", onepiece['colorNo'], True)
onepiece.insert(8, "colorBlack", onepiece['colorNo'], True)
onepiece.insert(8, "colorGrey", onepiece['colorNo'], True)
onepiece.insert(8, "colorWhite", onepiece['colorNo'], True)

In [252]:
onepiece['colorWhite'] = onepiece['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
onepiece['colorGrey'] = onepiece['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
onepiece['colorBlack'] = onepiece['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
onepiece['colorRed'] = onepiece['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
onepiece['colorPink'] = onepiece['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
onepiece['colorOrange'] = onepiece['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
onepiece['colorIvory'] = onepiece['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
onepiece['colorYellow'] = onepiece['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
onepiece['colorGreen'] = onepiece['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
onepiece['colorBlue'] = onepiece['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
onepiece['colorPurple'] = onepiece['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
onepiece['colorBrown'] = onepiece['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
onepiece['colorBeige'] = onepiece['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
onepiece['colorJean'] = onepiece['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
onepiece['colorPattern'] = onepiece['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 64, 69, 66] else 0)
onepiece['colorOthers'] = onepiece['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [253]:
print(onepiece.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [254]:
onepiece = onepiece.drop('colorNo', axis=1)

In [255]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(7, 23):
    num += onepiece[onepiece.columns[i]].value_counts()[1]
    print(onepiece[onepiece.columns[i]].value_counts())
print(num)
print(onepiece.shape)

0    8744
1     704
Name: colorWhite, dtype: int64
0    8986
1     462
Name: colorGrey, dtype: int64
0    7389
1    2059
Name: colorBlack, dtype: int64
0    9204
1     244
Name: colorRed, dtype: int64
0    8985
1     463
Name: colorPink, dtype: int64
0    9329
1     119
Name: colorOrange, dtype: int64
0    8900
1     548
Name: colorIvory, dtype: int64
0    9106
1     342
Name: colorYellow, dtype: int64
0    8759
1     689
Name: colorGreen, dtype: int64
0    7966
1    1482
Name: colorBlue, dtype: int64
0    9212
1     236
Name: colorPurple, dtype: int64
0    9128
1     320
Name: colorBrown, dtype: int64
0    8570
1     878
Name: colorBeige, dtype: int64
0    9352
1      96
Name: colorJean, dtype: int64
0    9191
1     257
Name: colorPattern, dtype: int64
0    8899
1     549
Name: colorOthers, dtype: int64
9448
(9448, 42)


In [256]:
onepiece.insert(23, "colorName", True)

In [257]:
onepiece['colorName'] = onepiece.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [258]:
onepiece['colorName'].value_counts()

black      2059
blue       1482
beige       878
white       704
green       689
others      549
ivory       548
pink        463
grey        462
yellow      342
brown       320
pattern     257
red         244
purple      236
orange      119
jean         96
Name: colorName, dtype: int64

### Price 전처리

In [259]:
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace(",", ""))
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace("원", ""))
onepiece['clothPrice']

0        98000
1       127000
2       269000
3       125300
4       164000
         ...  
9443    111200
9444     63750
9445     89500
9446     85000
9447     71100
Name: clothPrice, Length: 9448, dtype: object

In [260]:
onepiece['clothPrice'] = onepiece['clothPrice'].astype('int')

In [261]:
onepiece['clothReviewNo'] = onepiece['clothReviewNo'].apply(lambda x: x.replace(",", ""))
onepiece['clothReviewNo'] = onepiece['clothReviewNo'].astype('int')
onepiece['clothReviewNo']

0       0
1       0
2       0
3       0
4       0
       ..
9443    0
9444    5
9445    0
9446    0
9447    2
Name: clothReviewNo, Length: 9448, dtype: int32

In [262]:
onepiece.index = onepiece['clothId']
onepiece = onepiece.drop('clothId', True)
onepiece

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_18532\466132366.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  onepiece = onepiece.drop('clothId', True)


,largeCategory,largeCategoryName,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
clothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1475941,4,원피스,0,1,0,미디 원피스,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,린넨 브이 넥 슬리브리스 원피스,섹 바이 리군,//image.msscdn.net/images/goods_img/20200608/1...,98000,20200608,"['나시원피스', '린넨', '시원한']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
1995082,4,원피스,0,0,1,맥시 원피스,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,Flora Strap Shirt One-Piece Orange,마나비스세븐,//image.msscdn.net/images/goods_img/20210614/1...,127000,20210614,"['추천원피스', '여름원피스', '롱원피스', '플라워원피스', '나시원피스', ...",0,1,0.0,0,3,1,1,2,1,0,1,0,0
1723870,4,원피스,0,1,0,미디 원피스,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,20F EMILLY DRESS(에밀리)_01,랭앤루,//image.msscdn.net/images/goods_img/20201216/1...,269000,20201216,"['여자원피스', '여성원피스', '20대', '30대', '옐로우원피스']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
2197982,4,원피스,0,1,0,미디 원피스,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,J체크프릴셔츠원피스 JVOP721A,쥬시쥬디,//image.msscdn.net/images/goods_img/20211026/2...,125300,20211026,"['원피스', '롱원피스', '셔츠원피스', '프릴원피스', '체크원피스']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
2247825,4,원피스,0,0,1,맥시 원피스,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,Moidew orange tassel dress,쎄무아듀,//image.msscdn.net/images/goods_img/20211125/2...,164000,20211125,"['데일리룩', '여성복', '드레스', '원피스']",0,1,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324195,4,원피스,0,1,0,미디 원피스,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,반팔 볼륨소매 원피스 HWOP320B,미카이브,//image.msscdn.net/images/goods_img/20220124/2...,111200,20220124,"['원피스', '롱원피스', '미디원피스', '데일리룩', '여성원피스', '여성'...",0,1,0.0,0,0,0,0,0,0,0,0,0,0
1464994,4,원피스,0,0,1,맥시 원피스,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,로맨틱 오션 롱 원피스,레터프롬문,//image.msscdn.net/images/goods_img/20200527/1...,63750,20200527,"['나시원피스', '롱원피스', '여름원피스', '휴양지원피스']",0,1,5.0,5,0,0,0,0,0,0,0,0,0
2352427,4,원피스,0,1,0,미디 원피스,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,others,J세일러카라플리츠원피스 JVOP626B,쥬시쥬디,//image.msscdn.net/images/goods_img/20220210/2...,89500,20220210,"['오버핏드레스', '롱드레스', '미디원피스', '원피스', '플레어원피스', '...",0,1,0.0,0,0,0,0,0,0,0,0,0,0


In [263]:
onepiece.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9448 entries, 1475941 to 1503801
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   largeCategory           9448 non-null   int64  
 1   largeCategoryName       9448 non-null   object 
 2   smallCategoryMinidress  9448 non-null   int64  
 3   smallCategoryMidi       9448 non-null   int64  
 4   smallCategoryMaxidress  9448 non-null   int64  
 5   smallCategoryName       9448 non-null   object 
 6   colorWhite              9448 non-null   int64  
 7   colorGrey               9448 non-null   int64  
 8   colorBlack              9448 non-null   int64  
 9   colorRed                9448 non-null   int64  
 10  colorPink               9448 non-null   int64  
 11  colorOrange             9448 non-null   int64  
 12  colorIvory              9448 non-null   int64  
 13  colorYellow             9448 non-null   int64  
 14  colorGreen              9448 no

# Skirt

In [51]:
skirt = pd.read_csv('./musinsa_clothes_skirt.csv')

In [52]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [53]:
skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8030 entries, 0 to 8029
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            8030 non-null   int64  
 1   largeCategory      8030 non-null   int64  
 2   largeCategoryName  8030 non-null   object 
 3   smallCategory      8030 non-null   int64  
 4   smallCategoryName  8030 non-null   object 
 5   colorNo            8030 non-null   int64  
 6   clothName          8030 non-null   object 
 7   brand              8030 non-null   object 
 8   image              8030 non-null   object 
 9   clothPrice         8030 non-null   object 
 10  date               8030 non-null   int64  
 11  hashtags           8030 non-null   object 
 12  clothSexMen        8030 non-null   int64  
 13  clothSexWomen      8030 non-null   int64  
 14  clothSexCommon     8030 non-null   int64  
 15  clothRate          8030 non-null   float64
 16  clothReviewNo      8030 

In [54]:
skirt.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,8.030000e+03,8030.0,8030.00000,8030.000000,8.030000e+03,8030.0,8030.0,8030.0,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000
mean,1.827303e+06,5.0,501.82142,14.868742,2.020636e+07,0.0,1.0,0.0,1.441557,0.921046,0.747572,0.918929,1.466874,0.951930,0.224782,0.158157,0.232628,0.144085
std,4.721685e+05,0.0,0.80569,15.687411,1.121140e+04,0.0,0.0,0.0,2.164618,1.388257,1.216880,1.549058,2.183245,1.477841,0.417465,0.364911,0.422533,0.351197
min,1.465310e+05,5.0,501.00000,1.000000,2.014112e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.530896e+06,5.0,501.00000,2.000000,2.020073e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.954120e+06,5.0,502.00000,7.000000,2.021052e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.203965e+06,5.0,503.00000,24.000000,2.021103e+07,0.0,1.0,0.0,4.300000,2.000000,1.000000,1.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413365e+06,5.0,503.00000,71.000000,2.022031e+07,0.0,1.0,0.0,5.000000,5.000000,5.000000,5.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
skirt = skirt.drop_duplicates(subset = "clothId")
skirt = skirt.reset_index(drop=True)

In [56]:
skirt['smallCategory'] = skirt['smallCategory'].astype('str')

In [57]:
skirt['largeCategory'].unique()

array([5], dtype=int64)

In [58]:
skirt['largeCategoryName'].unique()

array(['스커트'], dtype=object)

In [59]:
skirt['smallCategory'].unique()

array(['503', '501', '502'], dtype=object)

In [60]:
skirt['smallCategoryName'].unique()

array(['롱스커트', '미니스커트', '미디스커트'], dtype=object)

In [61]:
skirt['smallCategory'] = skirt['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [62]:
skirt.insert(4, "smallCategoryLongskirt", skirt['smallCategory'], True)
skirt.insert(4, "smallCategoryMidi", skirt['smallCategory'], True)
skirt.insert(4, "smallCategoryMiniskirt", skirt['smallCategory'], True)

In [63]:
skirt['smallCategoryMiniskirt'] = skirt['smallCategoryMiniskirt'].apply(lambda x: 1 if x == 501 else 0)
skirt['smallCategoryMidi'] = skirt['smallCategoryMidi'].apply(lambda x: 1 if x == 502 else 0)
skirt['smallCategoryLongskirt'] = skirt['smallCategoryLongskirt'].apply(lambda x: 1 if x == 503 else 0)

In [64]:
skirt = skirt.drop('smallCategory', axis=1)

In [65]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [66]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 6):
    num += skirt[skirt.columns[i]].value_counts()[1]
    print(skirt[skirt.columns[i]].value_counts())
print(num)
print(skirt.shape)

0    4573
1    3449
Name: smallCategoryMiniskirt, dtype: int64
0    5466
1    2556
Name: smallCategoryMidi, dtype: int64
0    6005
1    2017
Name: smallCategoryLongskirt, dtype: int64
8022
(8022, 28)


### clothSexCommon 변환

In [67]:
skirt['clothSexMen'] = skirt.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
skirt['clothSexWomen'] = skirt.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [68]:
skirt = skirt.drop('clothSexCommon', axis=1)

In [69]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [70]:
skirt['clothSexMen'] = skirt.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
skirt['clothSexWomen'] = skirt.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [71]:
print(skirt['clothSexMen'].value_counts())
print(skirt['clothSexWomen'].value_counts())
print(skirt.shape)

0    8022
Name: clothSexMen, dtype: int64
1    8022
Name: clothSexWomen, dtype: int64
(8022, 27)


### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [72]:
skirt.insert(8, "colorOthers", skirt['colorNo'], True)
skirt.insert(8, "colorPattern", skirt['colorNo'], True)
skirt.insert(8, "colorJean", skirt['colorNo'], True)
skirt.insert(8, "colorBeige", skirt['colorNo'], True)
skirt.insert(8, "colorBrown", skirt['colorNo'], True)
skirt.insert(8, "colorPurple", skirt['colorNo'], True)
skirt.insert(8, "colorBlue", skirt['colorNo'], True)
skirt.insert(8, "colorGreen", skirt['colorNo'], True)
skirt.insert(8, "colorYellow", skirt['colorNo'], True)
skirt.insert(8, "colorIvory", skirt['colorNo'], True)
skirt.insert(8, "colorOrange", skirt['colorNo'], True)
skirt.insert(8, "colorPink", skirt['colorNo'], True)
skirt.insert(8, "colorRed", skirt['colorNo'], True)
skirt.insert(8, "colorBlack", skirt['colorNo'], True)
skirt.insert(8, "colorGrey", skirt['colorNo'], True)
skirt.insert(8, "colorWhite", skirt['colorNo'], True)

In [73]:
skirt['colorWhite'] = skirt['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
skirt['colorGrey'] = skirt['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
skirt['colorBlack'] = skirt['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
skirt['colorRed'] = skirt['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
skirt['colorPink'] = skirt['colorPink'].apply(lambda x: 1 if x in [47, 10, 45, 48, 55] else 0)
skirt['colorOrange'] = skirt['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
skirt['colorIvory'] = skirt['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
skirt['colorYellow'] = skirt['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
skirt['colorGreen'] = skirt['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
skirt['colorBlue'] = skirt['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
skirt['colorPurple'] = skirt['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
skirt['colorBrown'] = skirt['colorBrown'].apply(lambda x: 1 if x in [4, 27] else 0)
skirt['colorBeige'] = skirt['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
skirt['colorJean'] = skirt['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
skirt['colorPattern'] = skirt['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
skirt['colorOthers'] = skirt['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [74]:
print(skirt.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [75]:
skirt = skirt.drop('colorNo', axis=1)

In [76]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(7, 23):
    num += skirt[skirt.columns[i]].value_counts()[1]
    print(skirt[skirt.columns[i]].value_counts())
print(num)
print(skirt.shape)

0    7545
1     477
Name: colorWhite, dtype: int64
0    7465
1     557
Name: colorGrey, dtype: int64
0    6157
1    1865
Name: colorBlack, dtype: int64
0    7835
1     187
Name: colorRed, dtype: int64
0    7666
1     356
Name: colorPink, dtype: int64
0    7960
1      62
Name: colorOrange, dtype: int64
0    7576
1     446
Name: colorIvory, dtype: int64
0    7874
1     148
Name: colorYellow, dtype: int64
0    7400
1     622
Name: colorGreen, dtype: int64
0    7040
1     982
Name: colorBlue, dtype: int64
0    7848
1     174
Name: colorPurple, dtype: int64
0    7642
1     380
Name: colorBrown, dtype: int64
0    7181
1     841
Name: colorBeige, dtype: int64
0    7700
1     322
Name: colorJean, dtype: int64
0    7685
1     337
Name: colorPattern, dtype: int64
0    7756
1     266
Name: colorOthers, dtype: int64
8022
(8022, 42)


In [77]:
skirt.insert(23, "colorName", True)

In [78]:
skirt['colorName'] = skirt.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [79]:
skirt['colorName'].value_counts()

black      1865
blue        982
beige       841
green       622
grey        557
white       477
ivory       446
brown       380
pink        356
pattern     337
jean        322
others      266
red         187
purple      174
yellow      148
orange       62
Name: colorName, dtype: int64

### Price 전처리

In [80]:
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace(",", ""))
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace("원", ""))
skirt['clothPrice']

0       228000
1       156000
2       182000
3        88000
4       160000
         ...  
8017     99000
8018     89000
8019     32900
8020     51300
8021     58500
Name: clothPrice, Length: 8022, dtype: object

In [81]:
skirt['clothPrice'] = skirt['clothPrice'].astype('int')

In [82]:
skirt['clothReviewNo'] = skirt['clothReviewNo'].apply(lambda x: x.replace(",", ""))
skirt['clothReviewNo'] = skirt['clothReviewNo'].astype('int')
skirt['clothReviewNo']

0         0
1         0
2         0
3         0
4         0
       ... 
8017      1
8018      0
8019    148
8020     14
8021      0
Name: clothReviewNo, Length: 8022, dtype: int32

skirt.index = skirt['clothId']
skirt = skirt.drop('clothId', True)

### Index를 NewGoodsNo로 변경

In [83]:
sub_skirt = pd.read_csv('./skirt_size.csv', index_col = 'Unnamed: 0')
sub_skirt

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,387628,FREE/PINK,38762801116,1116,162.7,51.7,0.0,1.0,9,10,0,1,0,0,0,0,1,3,11,12
1,387631,FREE/BLUE,38763101604,1604,165.0,50.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0
2,387632,FREE/NAVY,38763201044,1044,163.0,50.0,0.0,1.0,2,0,0,0,0,0,0,0,0,0,0,0
3,407626,FREE,4076260740,740,165.0,52.0,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0
4,463655,FREE,4636550465,465,161.7,53.9,0.1,0.9,0,2,0,0,1,3,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,2389731,FREE,23897312219,2219,155.0,50.0,0.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0
2650,2391576,S,23915762318,2318,159.6,47.8,0.1,0.9,4,3,1,0,0,0,0,0,0,0,0,0
2651,2391598,S,23915981607,1607,167.3,65.6,0.0,1.0,0,0,0,0,1,4,0,0,4,6,1,1
2652,2394978,M,2394978371,371,164.0,52.2,0.0,1.0,0,0,0,0,0,0,0,0,0,0,3,1


In [84]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [85]:
skirt.insert(1, 'clothSize', 'FREE', True)
skirt.insert(1, 'newClothId', 0, True)
skirt.insert(34, 'avgSexWomen', 0, True)
skirt.insert(34, 'avgSexMen', 0, True)
skirt.insert(36, 'avgWeight', 0, True)
skirt.insert(36, 'avgHeight', 0, True)
skirt.insert(39, 'month12', 0, True)
skirt.insert(39, 'month11', 0, True)
skirt.insert(39, 'month10', 0, True)
skirt.insert(39, 'month9', 0, True)
skirt.insert(39, 'month8', 0, True)
skirt.insert(39, 'month7', 0, True)
skirt.insert(39, 'month6', 0, True)
skirt.insert(39, 'month5', 0, True)
skirt.insert(39, 'month4', 0, True)
skirt.insert(39, 'month3', 0, True)
skirt.insert(39, 'month2', 0, True)
skirt.insert(39, 'month1', 0, True)

In [86]:
goodsList = list(sub_skirt.goodsNo.unique())
for i in sub_skirt[sub_skirt.goodsNo == goodsList[3]].itertuples():
    idx = skirt[skirt.clothId==i.goodsNo].index[0]

In [87]:
newList = pd.DataFrame()
for id in goodsList:
    cnt = 0
    for i in sub_skirt[sub_skirt.goodsNo == id].itertuples():
        if cnt == 0:
            idx = skirt[skirt.clothId==i.goodsNo].index[0]
            skirt.at[idx, 'newClothId'] = i.newGoodsNo
            skirt.at[idx, 'clothSize'] = i.goodsSize
            skirt.at[idx, 'avgSexMen'] = i.userSexMen
            skirt.at[idx, 'avgSexWomen'] = i.userSexWomen
            skirt.at[idx, 'avgHeight'] = i.userHeight
            skirt.at[idx, 'avgWeight'] = i.userWeight
            skirt.at[idx, 'month1'] = i.month1
            skirt.at[idx, 'month2'] = i.month2
            skirt.at[idx, 'month3'] = i.month3
            skirt.at[idx, 'month4'] = i.month4
            skirt.at[idx, 'month5'] = i.month5
            skirt.at[idx, 'month6'] = i.month6
            skirt.at[idx, 'month7'] = i.month7
            skirt.at[idx, 'month8'] = i.month8
            skirt.at[idx, 'month9'] = i.month9
            skirt.at[idx, 'month10'] = i.month10
            skirt.at[idx, 'month11'] = i.month11
            skirt.at[idx, 'month12'] = i.month12
            cnt += 1
        else:
            sub = pd.DataFrame(skirt[skirt.clothId==id].iloc[0]).T
            sub.newClothId = i.newGoodsNo
            sub.clothSize = i.goodsSize
            sub.avgSexMen = i.userSexMen
            sub.avgSexWomen = i.userSexWomen
            sub.avgHeight = i.userHeight
            sub.avgWeight = i.userWeight
            sub.month1 = i.month1
            sub.month2 = i.month2
            sub.month3 = i.month3
            sub.month4 = i.month4
            sub.month5 = i.month5
            sub.month6 = i.month6
            sub.month7 = i.month7
            sub.month8 = i.month8
            sub.month9 = i.month9
            sub.month10 = i.month10
            sub.month11 = i.month11
            sub.month12 = i.month12
            skirt = pd.concat([skirt, sub])

In [88]:
skirt.columns

Index(['clothId', 'newClothId', 'clothSize', 'largeCategory',
       'largeCategoryName', 'smallCategoryMiniskirt', 'smallCategoryMidi',
       'smallCategoryLongskirt', 'smallCategoryName', 'colorWhite',
       'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange',
       'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple',
       'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers',
       'colorName', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'avgSexMen', 'avgSexWomen',
       'avgHeight', 'avgWeight', 'clothRate', 'month1', 'month2', 'month3',
       'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10',
       'month11', 'month12', 'clothReviewNo', 'fit', 'feeling', 'stretch',
       'visibility', 'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [89]:
skirt.shape

(8876, 61)

In [96]:
len(skirt[skirt.newClothId!=0])

8876

In [91]:
8876-8022

854

In [92]:
len(sub_skirt)

2654

In [93]:
len(sub_skirt.goodsNo.unique())

1800

In [94]:
skirt.newClothId = skirt.apply(lambda x: x.newClothId if x.newClothId!=0 else x.clothId, axis=1)

In [99]:
skirt.head()

,clothId,clothSize,largeCategory,largeCategoryName,smallCategoryMiniskirt,smallCategoryMidi,smallCategoryLongskirt,smallCategoryName,colorWhite,colorGrey,colorBlack,colorRed,colorPink,colorOrange,colorIvory,colorYellow,colorGreen,colorBlue,colorPurple,colorBrown,colorBeige,colorJean,colorPattern,colorOthers,colorName,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,avgSexMen,avgSexWomen,avgHeight,avgWeight,clothRate,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
newClothId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018790,2018790,FREE,5,스커트,0,0,1,롱스커트,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,티어드 셔링 맥시 스커트 - 마젠타,온느,//image.msscdn.net/images/goods_img/20210703/2...,228000,20210703,"['롱스커트', '맥시스커트', '핑크', '플레어스커트', '셔링']",0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,5,5,3,1,1,1,1
1844574,1844574,FREE,5,스커트,1,0,0,미니스커트,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,여성 주름 스커트 (형광 핑크),씨드느와,//image.msscdn.net/images/goods_img/20210315/1...,156000,20210315,"['봄스커트', '스커트추천', '여성골프웨어', '골프웨어', '여자골프웨어', ...",0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1672093,1672093,FREE,5,스커트,0,1,0,미디스커트,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,Stitched Wool Tweed Midi Skirt - Pink,온느,//image.msscdn.net/images/goods_img/20201030/1...,182000,20201030,"['하이웨스트', '미디스커트', 'H라인스커트', '트임', '트위드', '트위드...",0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,1,5,5,0,0,1,1
1687533,1687533,FREE,5,스커트,0,1,0,미디스커트,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,pink,써클 포켓 스커트_핑크&베이지,필로컬리,//image.msscdn.net/images/goods_img/20201112/1...,88000,20201112,"['겨울스커트', '미디스커트', '여성스커트', '출근룩', '데이트룩']",0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,1,5,4,1,0,1,1
2338446,2338446,FREE,5,스커트,0,1,0,미디스커트,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,orange,A라인 스커트 2182A754_800,오니츠카타이거,//image.msscdn.net/images/goods_img/20220203/2...,160000,20220203,['A라인스커트'],0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
skirt.index = skirt['newClothId']
skirt = skirt.drop('newClothId', axis=1)

In [100]:
skirt.to_csv('final_skirt.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')